# S&P PyTechnicalIndicators study
This notebook is a use case of how to use the [PyTechnicalIndicators]() package.

The data used is S&P 500 prices from the 13th of October 2022 to the 12th of October 2023.

The notebook will go through every function in the PyTechnicalIndicators package. The package is broken down into three parts, Bulk, Single, and Chart Patterns. The Bulk and Single functions are identical with the exception that Bulk functions will expect to have lists sent to them and will return lists. The Single functions will return singular data. The aim is for the bulk functions to be used on datasets of multiple days, and the single functions to increment on those. This will be demonstrated below.
The Chart Pattern functions are used to highlight what has happened on charts, their purpose is to be added to graphs. This will also be demonstrated below.

To use this notebook for your own data replace `example.csv` with your own data. The data is expected to be tab (`\t`) separated rather than comma (`,`) separated. But that can be changed on line 9 in the cell below.

## Formatting the raw data
The cell below will just prepare the CSV for us, print and chart out the data so that we have an initial view of what the data holds.

`data['Typical Price']` is used for a number of calculations in the PyTechnicalIndicators package.

Lines 13 to 18 will be used to add singular data to the bulk analysis that will be done on `data`

In [3]:
import datetime
import math

import pandas
import plotly.graph_objects as go
from plotly.subplots import make_subplots


data = pandas.read_csv("example.csv", sep='\t', index_col=0, parse_dates=True)
data.index.name = 'Date'
data['Typical Price'] = (data['High'] + data['Low'] + data['Close']) / 3
data.sort_index(inplace=True)

latest_date = datetime.date(2023, 10, 12)
latest_open = 4380.94
latest_high = 4382.67
latest_low = 4370.34
latest_close = 4370.56
latest_typical_price = (latest_high + latest_close + latest_low) / 3
latest_volume = 184806058

print(data)
# TODO: Remove weekends and bank holidays
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}]]
)

fig.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data['Typical Price'],
        name='Typical Price',
        line={'color': 'blue'}
    )
)
fig.add_trace(
    go.Bar(
        x=data.index,
        y=data['Volume'],
        name='Volume'
    ),
    row=2, col=1
)
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

               Open     High      Low    Close      Volume  Typical Price
Date                                                                     
2022-10-13  3520.37  3685.41  3491.58  3669.91  5021680000    3615.633333
2022-10-14  3690.41  3712.00  3579.68  3583.07  4243030000    3624.916667
2022-10-17  3638.65  3689.73  3638.65  3677.95  4352780000    3668.776667
2022-10-18  3746.26  3762.79  3686.53  3719.98  4483740000    3723.100000
2022-10-19  3703.11  3728.58  3666.51  3695.16  4223800000    3696.750000
...             ...      ...      ...      ...         ...            ...
2023-10-05  4259.31  4267.13  4225.91  4258.19  3581470000    4250.410000
2023-10-06  4234.79  4324.10  4219.55  4308.50  3902030000    4284.050000
2023-10-09  4289.02  4341.73  4283.79  4335.66  3174630000    4320.393333
2023-10-10  4339.75  4385.46  4339.64  4358.24  3520240000    4361.113333
2023-10-11  4366.59  4378.64  4345.34  4376.95  3601660000    4366.976667

[250 rows x 6 columns]


## The basic indicators
In this section we will cover the basic indicators.

A number of these don't have a single version as they are just calling functions in the `math` and `statisitics` packages.

In [4]:
from PyTechnicalIndicators.Bulk import basic_indicators as bulk_basic_indicators
from PyTechnicalIndicators.Single import basic_indicators as single_basic_indicator

### Mean
The mean (aka the average) simply tells you what the mean price the asset has been trading at for a given period. 

The period is determined by the function caller, it is the number of days the caller wants an average for. Common favorites are weekly, monthly, quarterly averages. Keeping in mind that we are looking at the S&P, which isn't traded on weekends, so a week is 5 days. If looking at cryptocurrencies the function call would need to have a period of 7. The below call also doesn't take into account bank holidays, it could be refined further but serves its purpose as an example.

The price used for the mean could be either Open, High, Low, Close, or Typical Price. The most commonly used is the typical price.

In [19]:
weekly_mean = bulk_basic_indicators.mean(data['Typical Price'], 5)
monthly_mean = bulk_basic_indicators.mean(data['Typical Price'], 20)
quarterly_mean = bulk_basic_indicators.mean(data['Typical Price'], 60)

# print(weekly_mean)

import statistics
weekly_mean.append(statistics.mean(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_mean.append(statistics.mean(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_mean.append(statistics.mean(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

fig_mean = make_subplots(
    rows=1,
    cols=1,
    specs=[[{"type": "candlestick"}]]
)
fig_mean.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_mean.add_trace(
    go.Scatter(
        x=data.index[-len(weekly_mean):],
        y=weekly_mean,
        name='Weekly Mean',
        line={'color': '#FFE4C4'}
    )
)
fig_mean.add_trace(
    go.Scatter(
        x=data.index[-len(monthly_mean):],
        y=monthly_mean,
        name='Monthly Mean',
        line={'color': '#E9967A'}
    )
)
fig_mean.add_trace(
    go.Scatter(
        x=data.index[-len(quarterly_mean):],
        y=quarterly_mean,
        name='Quarterly Mean',
        line={'color': '#FF7F50'}
    )
)
fig_mean.update_layout(xaxis_rangeslider_visible=False)
fig_mean.show()

[3665.8353333333334, 3679.923333333333, 3698.8106666666667, 3721.704, 3745.1773333333335, 3775.2153333333335, 3802.7366666666667, 3833.1820000000002, 3851.7926666666667, 3857.812, 3849.2780000000002, 3829.117333333333, 3806.5166666666664, 3790.287333333333, 3781.094, 3774.306, 3814.7033333333334, 3858.9366666666665, 3894.748, 3928.078, 3967.0933333333332, 3969.2826666666665, 3965.4146666666666, 3960.2586666666666, 3959.7653333333333, 3970.664, 3988.8953333333334, 3992.28, 3994.0646666666667, 4002.8953333333334, 4014.112, 4020.6393333333335, 4027.5593333333336, 4026.8333333333335, 4007.858, 3984.23, 3961.9386666666664, 3953.9913333333334, 3970.8233333333333, 3984.194, 3974.8633333333332, 3956.6459999999997, 3927.045333333333, 3883.168, 3856.036, 3836.4739999999997, 3830.904, 3832.008, 3829.1306666666665, 3822.808666666667, 3825.4453333333336, 3826.0653333333335, 3829.584, 3832.1133333333332, 3838.6406666666667, 3855.5333333333333, 3870.1626666666666, 3891.9173333333333, 3923.13266666666

### Median
The median is the middle value of series of prices. 

The median is an alternative to the mean, it is believed to be more significant than the mean, as the mean includes prices that could be seen as extremes. Whereas the median returns the middle value where most the prices are most likely to be around.

Similarily to the mean, the median needs a list of prices and a period. We will once again use the Typical Price and get the weekly, monthly, and quarterly medians


In [20]:
weekly_median = bulk_basic_indicators.median(data['Typical Price'], 5)
monthly_median = bulk_basic_indicators.median(data['Typical Price'], 20)
quarterly_median = bulk_basic_indicators.median(data['Typical Price'], 60)

# print(weekly_median)

weekly_median.append(statistics.median(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_median.append(statistics.median(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_median.append(statistics.median(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

fig_median = make_subplots(
    rows=1,
    cols=1,
    specs=[[{"type": "candlestick"}]]
)
fig_median.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_median.add_trace(
    go.Scatter(
        x=data.index[-len(weekly_mean):],
        y=weekly_mean,
        name='Weekly Median',
        line={'color': '#FFE4C4'}
    )
)
fig_median.add_trace(
    go.Scatter(
        x=data.index[-len(monthly_mean):],
        y=monthly_mean,
        name='Monthly Median',
        line={'color': '#E9967A'}
    )
)
fig_median.add_trace(
    go.Scatter(
        x=data.index[-len(quarterly_mean):],
        y=quarterly_mean,
        name='Quarterly Median',
        line={'color': '#FF7F50'}
    )
)
fig_median.update_layout(xaxis_rangeslider_visible=False)
fig_median.show()

[3668.7766666666666, 3686.0733333333337, 3696.75, 3719.353333333333, 3719.353333333333, 3783.2433333333333, 3823.6800000000003, 3840.4666666666667, 3846.94, 3870.5633333333335, 3870.5633333333335, 3870.5633333333335, 3804.27, 3795.15, 3795.15, 3770.33, 3795.15, 3824.596666666667, 3924.8633333333332, 3974.2066666666665, 3974.2066666666665, 3974.2066666666665, 3965.406666666667, 3960.403333333333, 3960.403333333333, 3960.403333333333, 3988.78, 3988.78, 3988.78, 4019.9, 4026.966666666667, 4032.933333333334, 4032.933333333334, 4032.933333333334, 4011.9266666666663, 3957.8433333333337, 3953.72, 3953.72, 3957.8433333333337, 3972.19, 3972.19, 3972.19, 3911.19, 3857.06, 3857.06, 3824.1866666666665, 3824.1866666666665, 3829.2100000000005, 3829.2100000000005, 3829.2100000000005, 3829.2100000000005, 3829.7066666666665, 3832.31, 3832.31, 3832.31, 3847.2999999999997, 3870.2766666666666, 3905.4566666666665, 3911.0266666666666, 3956.0733333333337, 3972.83, 3972.83, 3972.83, 3956.536666666667, 3956.53

### Standard Deviation
The standard deviation shows how far from the average.

This helps determine how volatile the market is. The further prices tend to be from the average the more volatile the market is.

In [36]:
weekly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 5)
monthly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 20)
quarterly_standard_deviation = bulk_basic_indicators.standard_deviation(data['Typical Price'], 60)

# print(weekly_standard_deviation)

weekly_standard_deviation.append(statistics.stdev(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_standard_deviation.append(statistics.stdev(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_standard_deviation.append(statistics.stdev(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

# TODO: Should be chart agaisnt the mean
fig_stdev = make_subplots(
    rows=4,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]]
)
fig_stdev.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_stdev.add_trace(
    go.Bar(
        x=data.index[-len(weekly_standard_deviation)-1:],
        y=weekly_standard_deviation,
        name='Weekly Standard Deviation'
    ),
    row=2, col=1
)
fig_stdev.add_trace(
    go.Bar(
        x=data.index[-len(monthly_standard_deviation)-1:],
        y=monthly_standard_deviation,
        name='Monthly Standard Deviation'
    ),
    row=3, col=1
)
fig_stdev.add_trace(
    go.Bar(
        x=data.index[-len(quarterly_standard_deviation)-1:],
        y=quarterly_standard_deviation,
        name='Quarterly Standard Deviation'
    ),
    row=4, col=1
)
fig_stdev.update_layout(xaxis_rangeslider_visible=False)
fig_stdev.show()


### Variance
The variance shows how spread out the prices from one another.

This helps determine how volatile the market is. The more spread out they are, the more volatile the market.

In [37]:
weekly_variance = bulk_basic_indicators.variance(data['Typical Price'], 5)
monthly_variance = bulk_basic_indicators.variance(data['Typical Price'], 20)
quarterly_variance = bulk_basic_indicators.variance(data['Typical Price'], 60)

# print(weekly_variance)

weekly_variance.append(statistics.variance(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_variance.append(statistics.variance(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_variance.append(statistics.variance(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

# TODO: Should be chart agaisnt the mean and standard dev
fig_var = make_subplots(
    rows=4,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]]
)
fig_var.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_var.add_trace(
    go.Bar(
        x=data.index[-len(weekly_variance)-1:],
        y=weekly_variance,
        name='Weekly Variance'
    ),
    row=2, col=1
)
fig_var.add_trace(
    go.Bar(
        x=data.index[-len(monthly_variance)-1:],
        y=monthly_variance,
        name='Monthly Variance'
    ),
    row=3, col=1
)
fig_var.add_trace(
    go.Bar(
        x=data.index[-len(quarterly_variance)-1:],
        y=quarterly_variance,
        name='Quarterly Variance'
    ),
    row=4, col=1
)
fig_var.update_layout(xaxis_rangeslider_visible=False)
fig_var.show()

### Mean absolute deviation
The mean absolute deviation also measures how much the prices deviate from the mean but uses the absolute value of the difference as opposed to the difference.

This also helps to determine market volatility.

In [38]:
weekly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 5)
monthly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 20)
quarterly_mean_absolute_deviation = bulk_basic_indicators.mean_absolute_deviation(data['Typical Price'], 60)

# print(weekly_mean_absolute_deviation)

weekly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_mean_absolute_deviation.append(single_basic_indicator.mean_absolute_deviation(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

# TODO: Should be chart agaisnt the mean
fig_mad = make_subplots(
    rows=4,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]]
)
fig_mad.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_mad.add_trace(
    go.Bar(
        x=data.index[-len(weekly_mean_absolute_deviation)-1:],
        y=weekly_mean_absolute_deviation,
        name='Weekly Mean Absolute Deviation'
    ),
    row=2, col=1
)
fig_mad.add_trace(
    go.Bar(
        x=data.index[-len(monthly_mean_absolute_deviation)-1:],
        y=monthly_mean_absolute_deviation,
        name='Monthly Mean Absolute Deviation'
    ),
    row=3, col=1
)
fig_mad.add_trace(
    go.Bar(
        x=data.index[-len(quarterly_mean_absolute_deviation)-1:],
        y=quarterly_mean_absolute_deviation,
        name='Quarterly Mean Absolute Deviation'
    ),
    row=4, col=1
)
fig_mad.update_layout(xaxis_rangeslider_visible=False)
fig_mad.show()


### Median absolute deviation
Similar to the mean absolute deviation but instead of comparing how much the prices deviate from the mean, it compares how much the prices deviate from the median.

This also helps to determine market volatility.

In [39]:
weekly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 5)
monthly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 20)
quarterly_median_absolute_deviation = bulk_basic_indicators.median_absolute_deviation(data['Typical Price'], 60)

# print(weekly_median_absolute_deviation)

weekly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_median_absolute_deviation.append(single_basic_indicator.median_absolute_deviation(data['Typical Price'][-59:].tolist()+[latest_typical_price]))


# TODO: Should be chart agaisnt the median
fig_mdad = make_subplots(
    rows=4,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]]
)
fig_mdad.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_mdad.add_trace(
    go.Bar(
        x=data.index[-len(weekly_median_absolute_deviation)-1:],
        y=weekly_median_absolute_deviation,
        name='Weekly Median Absolute Deviation'
    ),
    row=2, col=1
)
fig_mdad.add_trace(
    go.Bar(
        x=data.index[-len(monthly_median_absolute_deviation)-1:],
        y=monthly_median_absolute_deviation,
        name='Monthly Median Absolute Deviation'
    ),
    row=3, col=1
)
fig_mdad.add_trace(
    go.Bar(
        x=data.index[-len(quarterly_median_absolute_deviation)-1:],
        y=quarterly_median_absolute_deviation,
        name='Quarterly Median Absolute Deviation'
    ),
    row=4, col=1
)
fig_mdad.update_layout(xaxis_rangeslider_visible=False)
fig_mdad.show()

### Mode absolute deviation
Similar to the mean absolute deviation but instead of comparing how much the prices deviate from the mean, it compares how much the prices deviate from the mode (most occurring value).

This also helps to determine market volatility.

In [41]:
weekly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 5)
monthly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 20)
quarterly_mode_absolute_deviation = bulk_basic_indicators.mode_absolute_deviation(data['Typical Price'], 60)

# print(weekly_mode_absolute_deviation)

weekly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-4:].tolist()+[latest_typical_price]))
monthly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-19:].tolist()+[latest_typical_price]))
quarterly_mode_absolute_deviation.append(single_basic_indicator.mode_absolute_deviation(data['Typical Price'][-59:].tolist()+[latest_typical_price]))

# TODO: Should be chart agaisnt the mode
fig_mead = make_subplots(
    rows=4,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "bar"}], [{"type": "bar"}], [{"type": "bar"}]]
)
fig_mead.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_mead.add_trace(
    go.Bar(
        x=data.index[-len(weekly_mode_absolute_deviation)-1:],
        y=weekly_mode_absolute_deviation,
        name='Weekly Mode Absolute Deviation'
    ),
    row=2, col=1
)
fig_mead.add_trace(
    go.Bar(
        x=data.index[-len(monthly_mode_absolute_deviation)-1:],
        y=monthly_mode_absolute_deviation,
        name='Monthly Mode Absolute Deviation'
    ),
    row=3, col=1
)
fig_mead.add_trace(
    go.Bar(
        x=data.index[-len(quarterly_mode_absolute_deviation)-1:],
        y=quarterly_mode_absolute_deviation,
        name='Quarterly Mode Absolute Deviation'
    ),
    row=4, col=1
)
fig_mead.update_layout(xaxis_rangeslider_visible=False)
fig_mead.show()

### Logarithm
The log of the price 

TODO: Figure why it is used

In [43]:
log = bulk_basic_indicators.log(data['Typical Price'])

# print(log)

log.append(math.log(latest_typical_price))
# TODO: Overlay two scatters but on different Y axis
fig_log = make_subplots(
    rows=2,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "scatter"}]]
)
fig_log.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_log.add_trace(
    go.Scatter(
        x=data.index[-len(log)-1:],
        y=log,
        name='Logged Typical Price'
    ),
    row=2, col=1
)
fig_log.update_layout(xaxis_rangeslider_visible=False)
fig_log.show()

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/basic_indicators.py:18: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



### Log difference
The log difference measure the difference in price from one date to another.

It is another way to determine how volatile an asset is

In [44]:
log_diff = bulk_basic_indicators.log_diff(data['Typical Price'])

# print(log_diff)

log_diff.append(math.log(data['Typical Price'][-1]) - math.log(latest_typical_price))

# TODO: Overlay two scatters but on different Y axis
fig_log_diff = make_subplots(
    rows=2,
    cols=1,
    specs=[[{"type": "candlestick"}], [{"type": "Bar"}]]
)
fig_log_diff.add_trace(
    go.Candlestick(
        x=data.index,
        open=data['Open'],
        low=data['Low'],
        high=data['High'],
        close=data['Close'],
        name='S&P 500'
    ),
    row=1, col=1
)
fig_log_diff.add_trace(
    go.Bar(
        x=data.index[-len(log)-1:],
        y=log_diff,
        name='Typical Price Log Difference'
    ),
    row=2, col=1
)
fig_log_diff.update_layout(xaxis_rangeslider_visible=False)
fig_log_diff.show()

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/basic_indicators.py:38: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_1870/3197927883.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## Moving Averages
One of the most common indicators when looking at stock prices

In [348]:
from PyTechnicalIndicators.Bulk import moving_averages as bulk_moving_average
from PyTechnicalIndicators.Single import moving_averages as single_moving_average

### Moving Average
The moving average provides a general overview of the direction of the price without the daily fluctuations by taking the average of a given period.

The period is determined by the caller.

Similar as above, we will be doing a weekly, monthly, and quarterly moving average.

In [349]:
weekly_ma = bulk_moving_average.moving_average(data['Typical Price'], 5)
monthly_ma = bulk_moving_average.moving_average(data['Typical Price'], 20)
quarterly_ma = bulk_moving_average.moving_average(data['Typical Price'], 60)

print(weekly_ma)

weekly_ma.append(single_moving_average.moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_ma.append(single_moving_average.moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_ma.append(single_moving_average.moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

[3665.8353333333334, 3679.9233333333336, 3698.8106666666667, 3721.704, 3745.177333333333, 3775.2153333333335, 3802.7366666666667, 3833.182, 3851.7926666666667, 3857.8120000000004, 3849.278, 3829.117333333333, 3806.5166666666673, 3790.2873333333337, 3781.094, 3774.3059999999996, 3814.7033333333334, 3858.936666666667, 3894.7479999999996, 3928.078, 3967.0933333333332, 3969.282666666667, 3965.4146666666666, 3960.258666666667, 3959.7653333333337, 3970.6639999999998, 3988.895333333334, 3992.28, 3994.0646666666667, 4002.895333333333, 4014.112, 4020.6393333333335, 4027.5593333333336, 4026.8333333333335, 4007.858, 3984.2300000000005, 3961.9386666666664, 3953.991333333333, 3970.8233333333337, 3984.1940000000004, 3974.8633333333332, 3956.6459999999997, 3927.045333333333, 3883.168, 3856.036, 3836.4739999999997, 3830.904, 3832.0080000000003, 3829.130666666667, 3822.808666666667, 3825.445333333334, 3826.0653333333335, 3829.584, 3832.1133333333332, 3838.640666666666, 3855.533333333333, 3870.162666666

### Smoothed moving average
The smoothed moving average is similar to the moving average but puts more weight onto the more recent values when calculating the average

In [350]:
weekly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 5)
monthly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 20)
quarterly_sma = bulk_moving_average.smoothed_moving_average(data['Typical Price'], 60)

print(weekly_sma)

weekly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_sma.append(single_moving_average.smoothed_moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/moving_averages.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_sum += prices[i] * alpha_power


[3676.8704823100115, 3685.577359987307, 3701.5379946057433, 3729.036974456608, 3762.7635094399484, 3794.2389370141204, 3813.5406885610028, 3839.987209265429, 3856.319696969697, 3862.102168808504, 3846.3763747421863, 3811.8503871172456, 3790.18037283833, 3783.2643328573695, 3787.050088846581, 3780.397688402348, 3828.9799508170713, 3880.691377122005, 3916.8484118673646, 3947.9726701570676, 3970.475034110741, 3964.6090798032683, 3961.8827177534504, 3956.678540377598, 3962.8583880691735, 3979.5785784547043, 3997.9419102014917, 3995.4685038870375, 3988.7146263683962, 4001.862319530383, 4022.1533793431704, 4032.8247072822473, 4032.0178169125816, 4016.0044105981274, 3991.1665318102487, 3972.985913057274, 3957.153558622878, 3956.287420276059, 3980.8096335078526, 3992.1463953672846, 3971.407471045534, 3939.6590861494524, 3902.1376233539586, 3864.02352530541, 3851.845037283833, 3834.617766143106, 3830.821467555132, 3831.136582579724, 3824.328224654926, 3823.909319371727, 3825.7251991115336, 3827

### Exponential moving average
The exponential moving average is similar to the moving average but puts more weight onto the more recent values when calculating the average

In [351]:
weekly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 5)
monthly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 20)
quarterly_ema = bulk_moving_average.exponential_moving_average(data['Typical Price'], 60)

print(weekly_ema)

weekly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-4:] + [latest_typical_price]))
monthly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-19:] + [latest_typical_price]))
quarterly_ema.append(single_moving_average.exponential_moving_average(data['Typical Price'][-59:] + [latest_typical_price]))

[3684.39925750395, 3688.5182306477104, 3703.5706477093213, 3735.9148341232235, 3776.6986729857827, 3807.7049921011067, 3819.986413902054, 3844.879778830964, 3860.0561927330186, 3865.080047393366, 3842.6529383886264, 3797.6316113744083, 3778.9006477093217, 3780.2148973143767, 3792.6850710900485, 3783.517614533966, 3840.8438704581367, 3898.3243127962096, 3932.6702685624023, 3960.6203791469197, 3972.077503949448, 3960.5417219589267, 3959.517898894156, 3954.517567140602, 3965.8136808846766, 3986.597251184835, 4004.6619589257525, 3996.405718799369, 3983.8382464454985, 4002.435355450238, 4029.7865244865725, 4041.37533965245, 4033.308246445499, 4006.595323854661, 3978.9528120063196, 3965.9439968404436, 3954.377093206952, 3958.305924170616, 3989.0851500789895, 3997.739731437599, 3966.5313586097955, 3925.436208530806, 3884.2043601895743, 3851.2100473933656, 3850.365355450237, 3833.0923222748816, 3830.390315955767, 3830.441484992102, 3820.9359241706165, 3824.905971563982, 3826.124881516588, 3828

### Personalised moving average
The personalised moving average allows the caller to put weight on the prices they want. 

It essentially exposes the inner workings of the smoothed and exponential moving averages. The key parameters for the personalised moving average are the `alpha_nominator` and `alpha_denominator`. The alpha determines the weight that will be applied to prices. 

For the smoothed moving average the nominator is 1 and denominator is 0. For the exponential moving average the alpha nominator is 2 and the denominator is 1. They both call this function with hardcoded values.

In [352]:
weekly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 5, 4, 2)
monthly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 20, 4, 2)
quarterly_pma = bulk_moving_average.personalised_moving_average(data['Typical Price'], 60, 4, 2)

print(weekly_ema)

weekly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-4:] + [latest_typical_price], 4, 2))
monthly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-19:] + [latest_typical_price], 4, 2))
quarterly_pma.append(single_moving_average.personalised_moving_average(data['Typical Price'][-59:] + [latest_typical_price], 4, 2))

[3684.39925750395, 3688.5182306477104, 3703.5706477093213, 3735.9148341232235, 3776.6986729857827, 3807.7049921011067, 3819.986413902054, 3844.879778830964, 3860.0561927330186, 3865.080047393366, 3842.6529383886264, 3797.6316113744083, 3778.9006477093217, 3780.2148973143767, 3792.6850710900485, 3783.517614533966, 3840.8438704581367, 3898.3243127962096, 3932.6702685624023, 3960.6203791469197, 3972.077503949448, 3960.5417219589267, 3959.517898894156, 3954.517567140602, 3965.8136808846766, 3986.597251184835, 4004.6619589257525, 3996.405718799369, 3983.8382464454985, 4002.435355450238, 4029.7865244865725, 4041.37533965245, 4033.308246445499, 4006.595323854661, 3978.9528120063196, 3965.9439968404436, 3954.377093206952, 3958.305924170616, 3989.0851500789895, 3997.739731437599, 3966.5313586097955, 3925.436208530806, 3884.2043601895743, 3851.2100473933656, 3850.365355450237, 3833.0923222748816, 3830.390315955767, 3830.441484992102, 3820.9359241706165, 3824.905971563982, 3826.124881516588, 3828

### Moving Average Convergence Divergence
The Moving Average Convergence Divergence (MACD) is an oscillator that calculates the difference between the 12 and 26 period exponential moving averages. It is plotted against the Signal line a 9 period exponential moving average of the MACD.

It produces "buy" and "sell" signals when the two lines cross one another.

In [353]:
macd = bulk_moving_average.moving_average_convergence_divergence(data['Typical Price'])
signal = bulk_moving_average.signal_line(macd)

print(macd)
print(signal)

macd.append(single_moving_average.moving_average_convergence_divergence(data['Typical Price'][-26:] + [latest_typical_price]))
signal.append(single_moving_average.signal_line(macd[-10:]))

[53.25656001211337, 52.81985855578887, 52.49188950350117, 54.83163718401556, 58.49879379761251, 60.21593206997795, 57.24789366397181, 48.96199167853274, 47.935706493394264, 50.996870886203396, 50.73257783289819, 45.64623841637467, 36.788467430016226, 27.337154508155436, 21.514699910170293, 13.941294070439653, 8.04788290979468, 9.106979449114988, 7.9292446852346075, -1.2247057527733887, -16.04517175660021, -30.21082394870791, -40.72907057009479, -42.89159645673226, -47.719774478682666, -49.356296176829346, -50.79734655680477, -53.2275650845113, -52.1268012831888, -52.83828662523047, -51.199910858365456, -45.59711084113678, -42.57412475569208, -34.01820066163418, -21.92381733970933, -13.047291499935, 0.23633684826791068, 11.691379553412844, 21.160984738984553, 30.225888776816646, 32.92054578221041, 30.038578016288284, 31.975329091987533, 39.532049465991804, 45.68709179771486, 46.5476627348462, 49.781775846628534, 53.805082213859805, 51.347773506003705, 51.144042554049975, 53.340607906885

### Personalised MACD
The MACD makes a lot of assumptions that are no longer true. When it was invented the working week was 6 days, showing us how the two week moving average differed from the month moving average. As the work week is now 5 days (or 7 if trading cryptocurrencies) the personalised MACD allows the caller to personalise the MACD.

When calling `personalised_macd` the caller needs to provide the short period, the long period, and the moving average model they'd like to use.
When calling `personalised_signal` the caller needs to provide the period and the moving average model.

In [354]:
personalised_macd = bulk_moving_average.moving_average_convergence_divergence(data['Typical Price'], 10, 20, 'sma')
personalised_signal = bulk_moving_average.signal_line(data['Typical Price'], 15, 'sma')

print(personalised_macd)
print(personalised_signal)

personalised_macd.append(single_moving_average.moving_average_convergence_divergence(data['Typical Price'][-20:] + [latest_typical_price], 10, 20, 'sma'))
personalised_signal.append(single_moving_average.signal_line(personalised_macd[-15:], 'sma'))

[24.664624666289455, 26.26513903071327, 28.752751445681042, 30.588447529549285, 34.52657762167564, 38.15423955760252, 42.56747605669125, 46.877928160523425, 49.11754839212608, 53.80848141902425, 63.01370597053847, 62.36537569297843, 55.85248533676804, 48.62085295977431, 46.56430242631586, 47.016378286817144, 44.776646696722764, 38.105175693101955, 28.903580573412455, 17.941251714581085, 6.339488232787062, -1.095302529682158, -1.7142795596751057, 3.652883718878911, 2.1987079389546125, -9.280027627919935, -23.37949586454397, -34.88529034459452, -42.319222474659, -42.65967215397404, -47.0008085335985, -48.373940023554496, -51.70847417451523, -60.2798831445175, -61.0928497315158, -56.0018154108443, -48.024059618489446, -38.833264616202996, -33.88979297690321, -28.224480412389767, -16.423160157621624, -7.3981651554991, 5.150046170552287, 20.708875327838996, 32.63700293660122, 42.110442352206064, 45.33906926840564, 42.117349938593634, 43.98677093172182, 48.62782994163672, 48.58842778110056, 

### McGinley Dynamic
The McGinley Dynamic is an alternative to the moving average, it was built to be more resilient to shocks.

It uses previously calculated McGiley dynamics to determine the new one, so when calling the single function, if one is available it should be provided to get an accurate value

In [355]:
weekly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 5)
monthly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 20)
quarterly_mcginley_dynamic = bulk_moving_average.mcginley_dynamic(data['Typical Price'], 60)

print(weekly_mcginley_dynamic)

weekly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 5, weekly_mcginley_dynamic[-1]))
monthly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 20, monthly_mcginley_dynamic[-1]))
quarterly_mcginley_dynamic.append(single_moving_average.mcginley_dynamic(latest_typical_price, 60, quarterly_mcginley_dynamic[-1]))

[3615.633333333333, 3617.471053405242, 3627.1701221195917, 3644.4538246854554, 3654.333653179676, 3660.4657562602856, 3671.514917826437, 3691.335560047035, 3716.7919222652395, 3739.473804806434, 3754.8798037447277, 3775.5304092018087, 3793.6683919187567, 3807.8612061392273, 3807.1402490085216, 3788.7091674105395, 3782.487073042355, 3784.986026252466, 3792.585025697906, 3788.027995097555, 3811.7735777242133, 3840.045101169327, 3863.4335609863983, 3885.875733710786, 3900.5437294776784, 3907.347562326958, 3917.401429232644, 3923.413735753451, 3935.6508709845834, 3951.1319304534936, 3965.188267497656, 3967.5864465910017, 3965.5178921111, 3978.12178953574, 3995.881045207411, 4007.8439641534055, 4008.657185944689, 3997.0462176912633, 3984.5253645420707, 3979.043593782542, 3972.668494352055, 3972.572749361349, 3984.8094743677984, 3988.7494779059525, 3971.970084421657, 3946.124495302179, 3918.474487933991, 3896.3002812738036, 3890.7373423051586, 3873.971751920367, 3864.593408487091, 3857.35832

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/moving_averages.py:161: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_mcginley_dynamic = moving_averages.mcginley_dynamic(prices[0], period)


### Moving Average Envelopes
The moving average envelope function provides a moving average bracket calculated by taking a provided percentage and applying it to the calculated moving average.

`moving_average_envelope` needs to be given a list of prices, the period to calculate the moving average, as well as optionally the moving average model (defaults to moving average), and the difference (defaults to 3%). 

In [356]:
moving_average_envelope = bulk_moving_average.moving_average_envelopes(data['Typical Price'], 5, 'ema', 5)

print(moving_average_envelope)

moving_average_envelope.append(single_moving_average.moving_average_envelopes(data['Typical Price'][-4:] + [latest_typical_price], 'ema', 5))

[(3868.619220379148, 3684.39925750395, 3500.1792946287524), (3872.944142180096, 3688.5182306477104, 3504.0923191153247), (3888.7491800947873, 3703.5706477093213, 3518.3921153238553), (3922.7105758293847, 3735.9148341232235, 3549.1190924170623), (3965.533606635072, 3776.6986729857827, 3587.8637393364934), (3998.090241706162, 3807.7049921011067, 3617.3197424960513), (4010.9857345971573, 3819.986413902054, 3628.987093206951), (4037.123767772512, 3844.879778830964, 3652.6357898894157), (4053.0590023696695, 3860.0561927330186, 3667.0533830963677), (4058.3340497630343, 3865.080047393366, 3671.8260450236976), (4034.7855853080578, 3842.6529383886264, 3650.520291469195), (3987.5131919431287, 3797.6316113744083, 3607.750030805688), (3967.845680094788, 3778.9006477093217, 3589.9556153238555), (3969.2256421800957, 3780.2148973143767, 3591.2041524486576), (3982.319324644551, 3792.6850710900485, 3603.050817535546), (3972.6934952606643, 3783.517614533966, 3594.3417338072677), (4032.8860639810437, 384

## Oscillators
An oscillator in a technical indicator that oscillates between two predetermined limits.

In [357]:
from PyTechnicalIndicators.Bulk import oscillators as bulk_oscillators
from PyTechnicalIndicators.Single import oscillators as single_oscillators


### Stochastic Oscillator
The stochastic oscillator compares closing prices for the last 14 periods. It takes the difference between the latest close, lowest close, and highest close.
The asset is generally considered overbought over 80 and oversold under 20, but varies from asset to asset. 

In [358]:
stochastic_oscillator = bulk_oscillators.stochastic_oscillator(data['Close'])

print(stochastic_oscillator)

stochastic_oscillator.append(single_oscillators.stochastic_oscillator(data['Close'][-13:]+[latest_close]))

[85.86119060347808, 55.54262712663921, 22.998129887793155, 44.52992179530776, 59.9370962257736, 68.99438966337985, 15.830435502566804, 100.0, 100.0, 86.93231760914158, 99.56050395546447, 87.49633753296224, 83.01714034573693, 89.89525344271912, 84.25505420451223, 100.0, 100.0, 99.59094334206455, 77.27941440310016, 75.0152499192651, 100.0, 97.34930737551481, 93.70273305877926, 39.14638712092859, 0.0, 0.0, 20.240782543265706, 0.31465900540395114, 38.74410014364857, 58.64286202886654, 42.000136808263264, 0.0, 0.0, 0.0, 1.5294890116256736, 23.92536608408133, 2.341700084162589, 13.446210208426296, 5.73790781721874, 0.0, 27.940616673011178, 23.804085775916818, 17.307448293363795, 32.885431400282826, 22.109659646316597, 100.0, 97.32701591274827, 100.0, 100.0, 100.0, 100.0, 96.23847686107364, 67.46653078241539, 47.515576731766004, 86.1353997591497, 100.0, 98.64909546077175, 98.30428416229742, 100.0, 100.0, 69.25630423388273, 100.0, 100.0, 100.0, 84.59293011996704, 75.5508881848278, 94.389662169

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return ((close_prices[-1] - lowest_closing) / (highest_closing - lowest_closing)) * 100


### Personalised Stochastic Oscillator
The stochastic oscillator being fixed to 14 periods is rather inflexible as different assets trade on different periods, the personalised stochastic oscillator lets the function caller to determine their period
The single version does not need a period as it just assumes that the length of the submitted list is the period.

In [359]:
weekly_so = bulk_oscillators.stochastic_oscillator(data['Close'], 5)
monthly_so = bulk_oscillators.stochastic_oscillator(data['Close'], 20)
quarterly_so = bulk_oscillators.stochastic_oscillator(data['Close'], 60)

print(weekly_so)

weekly_so.append(single_oscillators.stochastic_oscillator(data['Close'][-4:]+[latest_close]))
monthly_so.append(single_oscillators.stochastic_oscillator(data['Close'][-19:]+[latest_close]))
quarterly_so.append(single_oscillators.stochastic_oscillator(data['Close'][-59:]+[latest_close]))

[81.87130231538954, 60.411949455847, 100.0, 100.0, 100.0, 85.25319402058645, 51.28807822489668, 100.0, 68.98464163822526, 52.047781569965714, 0.0, 0.0, 33.30922480110478, 63.80588796710982, 100.0, 26.501570874145465, 100.0, 100.0, 85.39859224095602, 99.50892126370935, 6.61925601750568, 0.0, 41.576267434138074, 7.482842594642691, 100.0, 100.0, 98.5256078634243, 18.10657009829277, 0.0, 100.0, 97.10973220117573, 93.1335728282166, 33.64630960156762, 0.0, 0.0, 21.476266511830598, 0.7085643869378249, 100.0, 100.0, 71.46710449161495, 0.0, 0.0, 0.0, 2.22897669706182, 77.8332693046486, 7.7821651859164245, 44.68575189207012, 13.428370292150804, 0.0, 100.0, 85.19527702088988, 61.943687556766314, 100.0, 0.0, 100.0, 96.56242814440127, 100.0, 100.0, 100.0, 100.0, 89.82965931863686, 0.0, 0.0, 73.58339984038312, 100.0, 97.6355820105819, 97.0320767195766, 100.0, 100.0, 2.8524107471479163, 100.0, 100.0, 100.0, 73.28230137662783, 33.42380767739426, 77.0529994175886, 9.871869539894753, 0.0, 10.86060606060

### Fast Stochastic
The fast stochastic is simply the moving average of the stochastic oscillator. 

The caller decides which moving average model they'd like to use and the period.

In [360]:
# Dropping the last value to demonstrate the use of the single function. This would obviously not be done normally
fast_stochastic = bulk_oscillators.fast_stochastic(weekly_so[:-1], 5, 'ema')

print(fast_stochastic)

fast_stochastic.append(single_oscillators.fast_stochastic(weekly_so[-5:], 'ema'))

[94.12240580088418, 91.33696638370171, 77.52609864136636, 85.01739909424425, 78.10522959851471, 66.99528880264593, 40.35372775048629, 24.309722918654852, 23.938143283318816, 36.96559037779624, 60.4011850382166, 50.44104551112916, 72.01598926650318, 84.71540644048814, 86.0347093483267, 90.5012878547792, 61.53550145181368, 35.96837537545873, 34.884182298482045, 21.811533536725126, 47.89918115649179, 69.98679029301636, 84.48048691068723, 61.16938618057305, 40.4013112246023, 60.26754081640154, 72.40215979369219, 79.0400476754584, 64.69438089901898, 43.12958726601265, 23.69776591825804, 19.133773379845348, 8.319682751686722, 42.234161475452815, 66.54473324271893, 71.7983946601735, 46.77990840160291, 31.15078560046512, 15.711898141226353, 6.274980020511123, 30.44957135221742, 23.287180491016496, 32.679032981014224, 26.940988781959952, 17.847977901549957, 46.352578288634, 63.21360358282855, 63.66273732654285, 80.15160711287876, 53.43440474191919, 68.95626982794612, 78.73296531092306, 87.74583

### Slow Stochastic
The slow stochastic is the moving average of the fast stochastic

The caller determines the moving average model and the period.

In [361]:
# Once again the last item is dropped to demonstrate how the single version is called
slow_stochastic = bulk_oscillators.slow_stochastic(fast_stochastic[:-1], 5, 'ema')

print(slow_stochastic)

slow_stochastic.append(single_oscillators.slow_stochastic(fast_stochastic[-5:], 'ema'))

[82.49501604588183, 75.957085252412, 61.51194770238164, 46.4209627853309, 35.838954727077876, 34.134770947044046, 42.55689109360909, 45.694885960018745, 56.88027825196086, 69.23112259447062, 77.31290555200715, 83.23068115392643, 76.5598117589124, 61.20702072980949, 49.91362723604284, 37.29959343426198, 38.6791283395468, 49.54225300795211, 63.64075999468898, 64.14576260045271, 57.170713067605384, 58.82824571233382, 63.47498772684923, 68.38831093953358, 67.33519971116345, 59.40458455697437, 45.65360267781206, 34.120786963571824, 21.945297759356265, 27.572822597495687, 41.74716354125036, 54.195867942252065, 53.12147435239309, 46.952090410061516, 35.19791506184751, 22.51012454552425, 23.06630295051266, 21.95230274183579, 25.605136859757152, 26.618084183751257, 24.27976428904157, 32.44131243419059, 44.717140192959306, 52.598656044844056, 64.47292533701582, 62.5924163533293, 65.85638351352225, 70.93312653808874, 77.75483354586612, 83.03714620644998, 90.89535689251838, 95.39952198715783, 96.5

### Slow Stochastic DS
The slow stochastic ds is the moving average of the slow stochastic

The caller determines the moving average model and the period

In [362]:
# Once again the last item is dropped to demonstrate how the single version is called
slow_stochastic_ds = bulk_oscillators.slow_stochastic_ds(slow_stochastic[:-1], 5, 'ema')

print(slow_stochastic_ds)

slow_stochastic_ds.append(single_oscillators.slow_stochastic_ds(slow_stochastic[-5:], 'ema'))

[51.02847271743562, 42.95248706701629, 41.13214431271834, 41.85345949727564, 47.39114804730842, 56.3592112403697, 65.52655690721697, 73.48411061651532, 76.06965653533108, 71.3341740777104, 63.21743594965109, 52.55536524109667, 45.67774173985312, 45.600095606422094, 51.7366830351748, 56.592518927646765, 57.78979348025065, 59.15454099509857, 61.29903036540521, 63.902126465525456, 65.20771958143756, 63.386269972429794, 56.80936416473779, 47.76256445878061, 36.808978842123096, 31.720155721659033, 34.169857423761094, 41.27703073792171, 46.18571888815195, 47.705342515238044, 43.92167073987289, 35.81200013186215, 29.989743915431816, 25.73490413791745, 24.612497488151945, 24.847290860462177, 24.74757579762301, 27.78608880519561, 34.580601666576484, 41.951221078483655, 51.37212868583714, 57.04904110905682, 61.67405141218608, 66.08570451583473, 71.2471324864024, 76.11561268032332, 82.47299044655904, 88.27532628577339, 92.31294831381992, 89.30346643505241, 79.73626194916005, 69.68106422522573, 63

### Visualizing the stochastics
The stochastics should be charted onto the same chart to get the full picture of the movement of the closing price

In [363]:
# TODO: Chart the stochastics together

### Money Flow Index
The money flow index is an oscillator that measures the relation between the price and the volume. It looks at the price and volume over the past 14 periods

The asset is generally considered overbought over 80 and oversold under 20, but this can vary from asset to asset

In [364]:
money_flow_index = bulk_oscillators.money_flow_index(data['Typical Price'], data['Volume'])

print(money_flow_index)

money_flow_index.append(single_oscillators.money_flow_index(data['Typical Price'][-13:]+[latest_typical_price], data['Volume'][-13:]+[latest_volume]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  raw_money_flow.append(typical_prices[i] * volume[i])


[46.7592500858178, 54.24295084627584, 47.003026664018634, 47.82077320349122, 47.64476126432493, 47.87768155907278, 40.09004466431703, 48.797971380883475, 40.39028712369125, 40.45733851852773, 48.308723854717854, 48.45713068570332, 41.24450537339881, 48.15937841968092, 40.95933184420242, 47.881181829183504, 40.44209368046862, 42.228461044977514, 41.35395374789369, 42.036922996184686, 43.075150057330866, 43.853280145330125, 44.23957428423368, 43.470344922432496, 51.66195907038892, 51.59459017829489, 43.90199995954593, 43.87078840163726, 43.87068255054779, 51.935915080965, 49.393265242420725, 42.41487279329826, 51.6923897399469, 42.792837422686006, 50.043646719997255, 50.414084838617676, 49.955006576991984, 43.53769682988426, 49.895536330325996, 56.46264715847778, 57.510048552243276, 51.00346682308999, 49.674143610821474, 58.583546776262004, 59.40769607663586, 56.302954996892396, 65.09437596211086, 56.7442159683439, 65.28465850347152, 65.76122676925769, 64.0722834372632, 65.04798859551934

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  raw_money_flow.append(typical_prices[i] * volume[i])


### Personalised Money Flow Index
The personalised money flow index is identical to the money flow index but allows the caller to determine the period for which the MFI is calculated.

In [365]:
weekly_personalised_mfi = bulk_oscillators.money_flow_index(data['Typical Price'], data['Volume'], 5)
monthly_personalised_mfi = bulk_oscillators.money_flow_index(data['Typical Price'], data['Volume'], 20)
quarterly_personalised_mfi = bulk_oscillators.money_flow_index(data['Typical Price'], data['Volume'], 60)

print(weekly_personalised_mfi)

weekly_personalised_mfi.append(single_oscillators.money_flow_index(data['Typical Price'][-4:]+[latest_typical_price], data['Volume'][-4:]+[latest_volume]))
monthly_personalised_mfi.append(single_oscillators.money_flow_index(data['Typical Price'][-19:]+[latest_typical_price], data['Volume'][-19:]+[latest_volume]))
quarterly_personalised_mfi.append(single_oscillators.money_flow_index(data['Typical Price'][-59:]+[latest_typical_price], data['Volume'][-59:]+[latest_volume]))

[51.309995407311504, 75.92312628894956, 76.95966876660196, 51.36511386252383, 75.06037517944266, 50.670652520775526, 25.473256749762626, 25.71897808352287, 25.807525745028954, 26.23769339218927, 51.88613064473324, 51.91902767748727, 52.97280202754111, 53.60566100541154, 52.94828061605991, 52.7319240715838, 54.254654172302125, 50.33705019470931, 28.154320309186616, 51.39474133892326, 26.022053679737326, 28.348507637317155, 52.58699469938735, 25.11794713585006, 50.27670577334441, 52.596254216442816, 30.54715890708927, 59.84308609197438, 29.668125208179887, 66.1836994040502, 55.741498384451525, 35.23357373216663, 55.714617405989316, 49.785753774304006, 51.44798852624126, 51.78129713587526, 26.695105754579956, 24.641146943657645, 53.57566532049003, 51.98833082290957, 50.37058377951366, 58.20102266001962, 36.32377432062341, 57.39124671906489, 59.696937663391665, 50.43243109493187, 54.401930695101015, 51.23970512090679, 78.06668373898728, 51.108835902627256, 50.438232094002935, 53.9863496817

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  raw_money_flow.append(typical_prices[i] * volume[i])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  raw_money_flow.append(typical_prices[i] * volume[i])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:20: FutureWarning: Series.__getitem__ tr

### Chaikin Oscillator
The Chaikin Oscillator calculates a long and short period of the accumulation period, takes their moving average, and returns the difference. The period for the long period is 10, and 3 for the short period.
This should be plotted against the `accumulation_distribution_indicator` function, where the Chaikin line being above the accumulation distribution indicator signifies a bull market, and being under is a bear market.

In [366]:
chaikin_oscillator = bulk_oscillators.chaikin_oscillator(data['High'], data['Low'], data['Close'], data['Volume'])

print(chaikin_oscillator)

chaikin_oscillator.append(single_oscillators.chaikin_oscillator(
    data['High'][-9:] + [latest_high],
    data['Low'][-9:] + [latest_low],
    data['Close'][-9:] + [latest_close],
    data['Volume'][-9:] + [latest_volume]))

# TODO: When charting chart against the accumulation distribution indicator

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[0], low[0], close[0], volume[0])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[i], low[i], close[i], volume[i], long_period_accumulation_distribution[-1])


[2697283302.3360415, 377615219.6881676, 425120213.65873337, -128802365.25517702, -1149112196.2217813, -3505813007.1405516, -4440471335.617295, -3357342715.3339486, -1042819242.9938507, 542179943.5181293, -409558256.80030155, 1026656991.0405838, 3013317171.7266707, 2250209409.3164663, 1378595818.6548421, -582707713.5876589, -191522928.33357048, 436300353.76384926, 1097394137.7712963, 2375870102.9611444, 3196132345.83241, 3139409575.871174, 1734811385.2059703, 744406340.28514, 2572371812.4724874, 3141975982.081339, 4026220271.87496, 2619201775.581356, 960354111.6953354, -560788285.6542277, -371469868.8844762, -1505374993.903933, -428826097.78274345, -1057577459.186018, -1372201475.0290809, -2532596434.0480704, -3098577947.3106766, -3428444011.302231, -2718299835.6706553, -1265869744.6848001, 118014375.5120039, 1615472083.4118218, 2062183702.885005, 956957028.7385473, 945268868.4106249, 1831974189.1519694, 1714657511.54185, 1762581673.1594067, 372712317.9699936, 894071185.0778666, -573016

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[0], low[0], close[0], volume[0])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[i], low[i], close[i], volume[i], long_period_accumulation_distribution[-1])


### Personalised Chaikin Oscillator
The personalised Chaikin Oscillator allows the caller to determine the long and short period, as well as which moving average model to use.

The single version doesn't require a long version as it assumes that the length of the submitted lists are the long period.

In [367]:
personalised_co = bulk_oscillators.chaikin_oscillator(data['High'], data['Low'], data['Close'], data['Volume'], 5, 20, 'ema')

print(personalised_co)

personalised_co.append(single_oscillators.chaikin_oscillator(
    data['High'][-19:] + [latest_high],
    data['Low'][-19:] + [latest_low],
    data['Close'][-19:] + [latest_close],
    data['Volume'][-19:] + [latest_volume], 5, 'ema'))

[-1949383765.5825143, -415127654.3951645, 1462126908.420606, 1228892383.8559287, 1041448756.298904, 312131527.7110369, 299363323.51619005, 502172194.7538486, 1021911266.847559, 2341686693.0830975, 3752446761.9280276, 4225953876.6818542, 3662385935.2013874, 3176717819.027276, 4361355001.010799, 4771088593.353893, 5543705676.115398, 5248720104.157129, 4361176704.708069, 2931697362.239105, 2333324185.5972424, 763103011.1138258, 971480565.2111621, 359598387.2680378, -393002218.3253794, -1729177214.3828812, -2776606180.5810146, -3915541661.305563, -4141739416.3313775, -3481849346.389227, -2482761188.09074, -961165571.2343185, 12043370.458591938, -208149979.71358836, 219588533.55370617, 1251960605.3435516, 1455662517.0678139, 1893616536.8922958, 1493383123.689049, 1935590840.9565532, 913358736.9658504, 1301545404.3447323, 2629171156.6894426, 4074983329.4189053, 5538955460.931602, 5715958142.237615, 4299144360.379831, 2721574172.494894, 2604672499.969309, 2702999006.215727, 3279612082.210005,

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[0], low[0], close[0], volume[0])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/oscillators.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accumulation_distribution = accumulation_distribution_indicator(high[i], low[i], close[i], volume[i], long_period_accumulation_distribution[-1])


### Williams %R
Williams %R is calculated by looking at the difference between the high, close, and low.

Prices between 0 and -20 are considered overbought, prices between -80 and -100 are considered oversold

In [368]:
weekly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 5)
monthly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 20)
quarterly_williams_percent_r = bulk_oscillators.williams_percent_r(data['High'], data['Low'], data['Close'], 60)

print(weekly_williams_percent_r)
# TODO: Either max here or do it in single
weekly_williams_percent_r.append(single_oscillators.williams_percent_r(latest_high, latest_low, latest_close))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/oscillators.py:148: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  williams_r.append(oscillators.williams_percent_r(max(high[i:j]), min(low[i:j]), close[j-1]))


[-24.936396150584454, -52.97908361094407, -8.087642983728028, -8.20475140827802, -1.7360627582044212, -23.268964939471445, -33.02894483307498, -2.662270257067917, -31.55312323079831, -51.564814814814866, -99.34034354385722, -89.82400299569377, -66.11121512825308, -49.14341883542401, -33.79183860614394, -68.73178294573638, -0.7856026293639177, -3.3234859675037605, -19.535410764872445, -13.038437214531685, -41.46197099733653, -67.27718724448084, -51.92150449713811, -64.51349141455435, -2.315280853634164, -5.124174787802554, -7.846642828764505, -69.60667461263412, -79.26740686935764, -0.0, -14.69974210978757, -17.690040525605045, -62.42785214294486, -87.44234570612767, -91.4726553920491, -72.16361280769924, -88.07250484857508, -13.173724735322699, -45.608032308727765, -62.910909957122335, -92.64141573743842, -91.04559604468038, -94.14462315565602, -89.92794607577285, -49.11213517665121, -54.20028476506877, -35.90521024495322, -48.328412989707154, -85.05545360248946, -9.509071504802378, -2

## Strength Indicators
Strength indicators show how strong the trend (price movement), or the momentum is.

In [369]:
from PyTechnicalIndicators.Bulk import strength_indicators as bulk_strength_indicators
from PyTechnicalIndicators.Single import strength_indicators as single_strength_indicators

### Relative Strength Index
The Relative Strength Index (RSI) shows how strong the price momentum of the asset is.

It looks at the 14 previous periods of the price.

In [370]:
rsi = bulk_strength_indicators.relative_strength_index(data['Typical Price'])

print(rsi)

rsi.append(single_strength_indicators.relative_strength_index(data['Typical Price'][-13:] + [latest_typical_price]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_indicators.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if prices[i] > prices[i - 1]:
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_indicators.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  previous_gains.append(prices[i] - prices[i - 1])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_indicators.py:27: FutureWarning: Series.__getitem__ treat

[69.82424652428142, 53.86070890801618, 44.55270605963076, 43.705683188225294, 42.5210402744929, 38.75039822288654, 39.00852269004652, 51.13953224583465, 51.10458057678744, 59.86431431540698, 55.148641594120285, 57.50703813992027, 60.3238049839347, 55.935256070721685, 62.36021798716005, 67.25010246525241, 66.13603256273726, 62.45244619525536, 55.78799725414659, 59.79564641271755, 59.53128388181943, 59.49000393923527, 60.06226847219624, 57.03715573668876, 52.21147779068839, 55.093333861770695, 52.64896008593431, 55.409115870332705, 52.97893129194642, 58.6370587185637, 59.934876555306396, 52.333528610802674, 50.05255558011336, 49.407530581987224, 54.004531745400314, 54.231200413031424, 52.6243223257676, 47.57736929034467, 39.52411177761586, 41.1300749795723, 42.118923758179676, 45.405058577379336, 34.640127906373834, 34.40190943012057, 38.18892580589927, 49.35513178470561, 52.51945197136097, 56.75821743708055, 59.00113220717959, 63.044523804108046, 61.09091603408066, 60.487174403314775, 5

### Personalised RSI
The personalised RSI allows the caller to determine the period rather than having it look at the 14 previous values. It also allows the caller to determine the moving average model being used.

For the single version the period is determined from the length of the submitted list.

In [371]:
# TODO: Reenable week once fixed
# weekly_rsi = bulk_strength_indicators.relative_strength_index(data['Typical Price'], 5, 'ema')
monthly_rsi = bulk_strength_indicators.relative_strength_index(data['Typical Price'], 20, 'ema')
quarterly_rsi = bulk_strength_indicators.relative_strength_index(data['Typical Price'], 60, 'ema')

# print(weekly_rsi)

# weekly_rsi.append(single_strength_indicators.relative_strength_index(data['Typical Price'][-4:] + [latest_typical_price], 'ema'))
monthly_rsi.append(single_strength_indicators.relative_strength_index(data['Typical Price'][-19:] + [latest_typical_price], 'ema'))
quarterly_rsi.append(single_strength_indicators.relative_strength_index(data['Typical Price'][-59:] + [latest_typical_price], 'ema'))

### Accumulation Distribution Indicator
The accumulation distribution indicator (ADI) shows the link between the price and the volume. By doing so it tries to determine the strength of the trend. A rising line in combination with rising confirms an upward trend.

The ADI uses the previously calculated ADI to get the new value, this is passable when calling the single function.

In [372]:
adi = bulk_strength_indicators.accumulation_distribution_indicator(data['High'], data['Low'], data['Close'], data['Volume'])

print(adi)

adi.append(single_strength_indicators.accumulation_distribution_indicator(
    latest_high,
    latest_low,
    latest_close,
    latest_volume,
    adi[-1]
))

[4218542817.93324, 192923212.43144703, 2538038827.153415, 1987713539.9779537, 1663119919.8715909, -1777732588.9267828, 2827742416.051955, 5733951124.982378, 10005764744.285358, 6201890127.661609, 2100485127.6616468, 6159668128.896711, 4116230191.089878, 1256402247.27458, -3569704558.8538666, -4359990569.53278, -2143139534.1042037, 937867958.2816477, 1602043636.6186266, -2496716201.1753263, 3054327025.155429, 6959574882.550377, 2545167823.3911457, 2641271953.1651525, -234609855.5305524, 2499642184.6452184, 3861366201.95339, 4471349893.509564, 7994344913.917699, 10056310107.539581, 9729431646.00106, 7159596799.983371, 7235738763.173568, 13815098763.173569, 13975609415.873024, 16679750173.53316, 14206751450.7551, 12242241773.180536, 10777495413.192032, 12553233160.845879, 8901911241.791788, 12784903559.063026, 10211103888.906437, 8750779047.231262, 6113203679.770561, 4436051203.58015, 3018229901.135763, 3895347934.92273, 5963344438.002401, 7168214985.484642, 9874238400.118786, 9750053207.

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/strength_indicators.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adi_list = [strength_indicators.accumulation_distribution_indicator(high[0], low[0], close[0], volume[0], 0)]
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/strength_indicators.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adi_list.append(strength_indicators.accumulation_distribution_indicator(high[i], low[i], close[i], volume[i], adi_list[-1]))


### Directional Indicator, Directional Index, Average Directional Index, Average Directional Index Rating
The next three indicators are grouped into one as they all depend on one another and essentially create a combined indicator.
These indicators try to give the caller an idea of the direction of the price as well as the strength of the trend
The directional indicator (DI) returns the positive and negative directional indicators, as well as the true range. This returns a list of tuples, so will need to be extracted before passing it to the DX.
The directional index (DX) takes the positive and negative DI as parameters.
The average directional index (ADX) takes the DX as a parameter as well as a period and a moving average model

In [373]:
# TODO: Improve the description, go to Welles book. Get Welles default, add explanation about period DI
weekly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 5)
weekly_positive_di = [di[0] for di in weekly_di]
weekly_negative_di = [di[1] for di in weekly_di]
weekly_dx = bulk_strength_indicators.directional_index(weekly_positive_di, weekly_negative_di)
weekly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 5, 'ema')
weekly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 5)

monthly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 20)
monthly_positive_di = [di[0] for di in monthly_di]
monthly_negative_di = [di[1] for di in monthly_di]
monthly_dx = bulk_strength_indicators.directional_index(monthly_positive_di, monthly_negative_di)
monthly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 20, 'ema')
monthly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 20)

quarterly_di = bulk_strength_indicators.directional_indicator(data['High'], data['Low'], data['Close'], 60)
quarterly_positive_di = [di[0] for di in weekly_di]
quarterly_negative_di = [di[1] for di in weekly_di]
quarterly_dx = bulk_strength_indicators.directional_index(quarterly_positive_di, quarterly_negative_di)
quarterly_adx = bulk_strength_indicators.average_directional_index(weekly_dx, 60, 'ema')
quarterly_adxr = bulk_strength_indicators.average_directional_index_rating(weekly_adx, 60)

weekly_di.append(single_strength_indicators.directional_indicator_known_previous(
    latest_high,
    data['High'][-1],
    latest_low,
    data['Low'][-1],
    data['Close'][-1],
    weekly_di[-1][2],
    weekly_di[-1][0],
    weekly_di[-1][1],
    5
))
weekly_dx.append(single_strength_indicators.directional_index(weekly_di[-1][0], weekly_di[-1][1]))
weekly_adx.append(single_strength_indicators.average_directional_index(weekly_dx[-5:], 'ema'))
weekly_adxr.append(single_strength_indicators.average_directional_index_rating(weekly_adx[-1], weekly_adx[-5]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_indicators.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tr = other_indicators.true_range(high[0], low[0], previous_close[0])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_indicators.py:83: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dm = directional_movement(high[i], high[i-1], low[i], low[i-1])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/strength_ind

## Momentum Indicators

In [374]:
from PyTechnicalIndicators.Bulk import momentum_indicators as bulk_momentum_indicators
from PyTechnicalIndicators.Single import momentum_indicators as single_momentum_indicators

### Rate of Change
The rate of change (RoC) returns the percentage change between the price and the previous price.
This can be done any period.

In [375]:
weekly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 5)
monthly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 20)
quarterly_roc = bulk_momentum_indicators.rate_of_change(data['Typical Price'], 60)

print(weekly_roc)

weekly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-4]))
monthly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-19]))
quarterly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-59]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/momentum_indicators.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rates_of_change.append(momentum_indicators.rate_of_change(current_close_price=closing_prices[i+period], previous_close_price=closing_prices[i]))


[1.9482063999852635, 2.6052093151566655, 3.1200227505444604, 3.1523909287063683, 4.062757827821737, 3.733150543215813, 4.092826172291606, 2.4596179821017756, 0.7836721231794783, -1.1091932808933873, -2.636291042486132, -2.918791122315271, -2.093407023370241, -1.1875962930460604, -0.8921553938074861, 5.425553537004987, 5.884319684845247, 4.7180392518521375, 4.357322210010806, 5.173994495618879, 0.2789056773951377, -0.4859609874338667, -0.6486829237197801, -0.061801909846035806, 1.3742180289200994, 2.3160840250588115, 0.4273133796978551, 0.22599719044209618, 1.1069382952515217, 1.395142499398828, 0.8104528636111573, 0.8699310592206803, -0.09172805033671799, -2.352547360068775, -2.8984416848286063, -2.7455063343627453, -0.9904634348583862, 2.128628228604956, 1.6976224313684247, -1.1787564439555964, -2.3070740364255165, -3.7259882667579736, -5.433214128866295, -3.387342037648783, -2.500773421899739, -0.722052547795457, 0.1443444183338326, -0.3767629117243477, -0.8169541900885218, 0.3457125

/tmp/ipykernel_1851/927717031.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  weekly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-4]))
/tmp/ipykernel_1851/927717031.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  monthly_roc.append(single_momentum_indicators.rate_of_change(latest_typical_price, data['Typical Price'][-19]))
/tmp/ipykernel_1851/927717031.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

### On Balance Volume
The on balance volume (OBV) tries to show buying and selling pressure on an asset. 
It is calculated by looking at the change in volume in comparison to the close price.
The single version takes the previous OBV as an optional parameter.

In [376]:
obv = bulk_momentum_indicators.on_balance_volume(data['Close'], data['Volume'])

print(obv)

obv.append(single_momentum_indicators.on_balance_volume(latest_close, data['Close'][-1], latest_volume, data['Volume'][-1]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/momentum_indicators.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obv_list = [volume[0]]
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/momentum_indicators.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obv_list.append(momentum_indicators.on_balance_volume(closing_prices[i], closing_prices[i-1], volume[i], obv_list[-1]))


[5021680000, 778650000, 5131430000, 9615170000, 5391370000, 894750000, 5972770000, 10720700000, 15563820000, 10746510000, 6059190000, 10518600000, 5697980000, 1216770000, -3682230000, -8307520000, -2907340000, 1434280000, 6041920000, 1396910000, 7178170000, 12771480000, 8209550000, 13224860000, 9059540000, 5007760000, 9045120000, 5194430000, 9082420000, 12362140000, 10655680000, 7040250000, 3494210000, 10073570000, 5546440000, 1533820000, -2747000000, -7115380000, -11233430000, -7226530000, -11114790000, -7210660000, -2131300000, -6603640000, -11097540000, -18591200000, -22560810000, -18575440000, -14800240000, -18757190000, -15937910000, -18968210000, -22051730000, -19048050000, -22027920000, -25987060000, -21572980000, -25466430000, -21542870000, -25854640000, -22003610000, -17700250000, -13259990000, -9320290000, -13555850000, -17854560000, -21846060000, -17832700000, -13887490000, -17207920000, -20931940000, -17122350000, -13214590000, -17016590000, -12337270000, -7480340000, -1855

/tmp/ipykernel_1851/1134212347.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obv.append(single_momentum_indicators.on_balance_volume(latest_close, data['Close'][-1], latest_volume, data['Volume'][-1]))


### Commodity Channel Index
The commodity channel Index (CCI) is a momentum indicator that takes the difference the latest price and the moving average and divides it by the absolute deviation.
The caller can determine the moving average model, absolute deviation model, and the period. These default the standards normally used to calculate the CCI.
The single version doesn't take a period, it is determined from the length of the price parameter.

In [377]:
weekly_cci = bulk_momentum_indicators.commodity_channel_index(data['Typical Price'], 5, 'ema', 'median')
monthly_cci = bulk_momentum_indicators.commodity_channel_index(data['Typical Price'], 20, 'ema', 'median')
quarterly_cci = bulk_momentum_indicators.commodity_channel_index(data['Typical Price'], 60, 'ema', 'median')

print(weekly_cci)

weekly_cci.append(single_momentum_indicators.commodity_channel_index(data['Typical Price'][-4:] + [latest_typical_price], 'ema', 'median'))
monthly_cci.append(single_momentum_indicators.commodity_channel_index(data['Typical Price'][-19:] + [latest_typical_price], 'ema', 'median'))
quarterly_cci.append(single_momentum_indicators.commodity_channel_index(data['Typical Price'][-59:] + [latest_typical_price], 'ema', 'median'))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/moving_averages.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_sum += prices[i] * alpha_power
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/momentum_indicators.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (typical_prices[-1] - moving_average) / (0.015 * absolute_deviation)


[22.96103828973759, -6.459950100078479, 60.053596225455124, 127.72155443142817, 88.24065768256737, 46.38036728242358, 6.661952091239784, 79.75214662635112, 64.65414201858384, 23.65830754613455, -106.68446936635304, -112.89047661206263, -25.52558469098398, 25.74396298414755, 72.15899868084794, -31.786575718198122, 126.98283540669884, 80.0281316097464, 38.564251259680645, 37.01553985393432, -27.547230272467463, -118.15269424291519, 4.262634504029769, -28.036365817883002, 109.44159692791683, 89.20458793872736, 53.861794549550915, -52.51018916910462, -78.70055692871723, 81.18289425052669, 88.37773348400823, 30.242386609068692, -42.854868977276894, -103.71371043635122, -55.912589502279964, -15.021814165912264, -24.852120224514685, 105.59035538355624, 131.31014510497874, 17.47141462573324, -100.55483430808088, -83.01912111265678, -55.33471646820381, -39.88578506818843, 45.694552240037986, -69.57863215164251, -5.908670183052956, -3.651271182287192, -68.85666504623347, 85.13189220496021, 3.984

## Trend Indicators

In [378]:
from PyTechnicalIndicators.Bulk import trend_indicators as bulk_trend_indicators
from PyTechnicalIndicators.Single import trend_indicators as single_trend_indicators

### Aroon Oscillator
The Aroon oscillator uses the aroon up and aroon down to determine the strength of the trend.
It oscilates between -100 and 100, anything above 0 could mean the beginning of an uptrend, or a downtrend if it goes under 0. Anything past -50 or 50 can signal big price movements.
By default it looks at 25 periods.

In [380]:
aroon_up = bulk_trend_indicators.aroon_up(data['High'])
aroon_down = bulk_trend_indicators.aroon_down(data['Low'])
aroon_oscillator = bulk_trend_indicators.aroon_oscillator(data['High'], data['Low'])

print(aroon_oscillator)

aroon_up.append(single_trend_indicators.aroon_up(data['High'][-26:] + [latest_high]))
aroon_up.append(single_trend_indicators.aroon_down(data['Low'][-26:] + [latest_low]))
aroon_up.append(single_trend_indicators.aroon_oscillator(data['High'][-26:] + [latest_high], data['Low'][-26:] + [latest_low]))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if highs[i] == local_max:
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if lows[i] == local_low:


[92.0, 88.0, 84.0, 68.0, 92.0, 96.0, 96.0, 60.0, 72.0, 76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 72.0, 92.0, 92.0, 92.0, -12.0, -16.0, -20.0, -20.0, -28.0, -28.0, -28.0, -28.0, -27.999999999999993, -28.0, -28.0, -28.0, -28.0, -28.0, -28.0, -28.000000000000004, -28.0, -28.0, -28.0, -28.0, -28.0, -28.0, -28.000000000000004, -24.0, 76.0, 76.0, 88.0, 92.0, 92.0, 92.0, 92.0, 96.0, 96.0, 84.0, 84.0, 84.0, 76.0, 76.0, 72.0, 64.0, 64.0, 40.0, 40.00000000000001, 40.0, 40.0, 40.0, 40.0, 36.0, -60.0, -72.0, -76.0, -76.0, -76.0, -76.0, -76.0, -96.0, -100.0, -100.0, -92.0, -92.0, -72.0, -72.0, -72.0, -72.0, -72.0, -68.0, -64.0, -60.0, -20.000000000000007, -20.0, -19.999999999999996, 56.0, 60.0, 64.0, 64.0, 64.0, 64.0, 64.0, 84.0, 88.0, 92.0, 92.0, 100.0, 92.0, 92.0, 88.0, 84.0, 64.0, 64.0, 64.0, 96.0, 100.0, 92.0, 92.0, 88.0, 84.0, -12.0, -12.0, -12.0, -12.0, -12.0, -12.0, -11.999999999999993, -12.0, 40.0, 43.99999999999999, 44.0, 44.0, 44.0, 44.0, 44.0, 68.0, 68.0, 76.0, 80.0, 84.0, 84.0, 84.0, 84

### Personalised Aroon Oscillator
To personalise the Aroon oscillator so that is more aligned with what the caller is interested in a period parameter can be passed in.

In [385]:
weekly_aroon_up = bulk_trend_indicators.aroon_up(data['High'], 5)
weekly_aroon_down = bulk_trend_indicators.aroon_down(data['Low'], 5)
weekly_aroon_oscillator = bulk_trend_indicators.aroon_oscillator(data['High'], data['Low'], 5)

print(weekly_aroon_up)

weekly_aroon_up.append(single_trend_indicators.aroon_up(data['High'][-6:] + [latest_high], 5))
weekly_aroon_down.append(single_trend_indicators.aroon_down(data['Low'][-6:] + [latest_low], 5))
weekly_aroon_oscillator.append(single_trend_indicators.aroon_oscillator(data['High'][-6:] + [latest_high], data['Low'][-6:] + [latest_low], 5))

monthly_aroon_up = bulk_trend_indicators.aroon_up(data['High'], 20)
monthly_aroon_down = bulk_trend_indicators.aroon_down(data['Low'], 20)
monthly_aroon_oscillator = bulk_trend_indicators.aroon_oscillator(data['High'], data['Low'], 20)

monthly_aroon_up.append(single_trend_indicators.aroon_up(data['High'][-21:] + [latest_high], 20))
monthly_aroon_down.append(single_trend_indicators.aroon_down(data['Low'][-21:] + [latest_low], 20))
monthly_aroon_oscillator.append(single_trend_indicators.aroon_oscillator(data['High'][-21:] + [latest_high], data['Low'][-21:] + [latest_low], 20))

quarterly_aroon_up = bulk_trend_indicators.aroon_up(data['High'], 60)
quarterly_aroon_down = bulk_trend_indicators.aroon_down(data['Low'], 60)
quarterly_aroon_oscillator = bulk_trend_indicators.aroon_oscillator(data['High'], data['Low'], 60)

quarterly_aroon_up.append(single_trend_indicators.aroon_up(data['High'][-61:] + [latest_high], 60))
quarterly_aroon_down.append(single_trend_indicators.aroon_down(data['Low'][-61:] + [latest_low], 60))
quarterly_aroon_oscillator.append(single_trend_indicators.aroon_oscillator(data['High'][-61:] + [latest_high], data['Low'][-61:] + [latest_low], 60))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if highs[i] == local_max:
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if lows[i] == local_low:


[60.0, 40.0, 100.0, 100.0, 100.0, 80.0, 100.0, 80.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 100.0, 100.0, 80.0, 60.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 0.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 0.0, 0.0, 40.0, 20.0, 0.0, 80.0, 100.0, 80.0, 60.0, 100.0, 100.0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 60.0, 40.0, 100.0, 80.0, 60.0, 100.0, 100.0, 80.0, 60.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 20.0, 0.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.0, 0.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 20.0, 0.0, 80.0, 60.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 100.0, 100.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 100.0, 100.0, 100.0, 80.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 0.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 0.0, 100.0, 80.0, 60.0, 40.0, 20.0, 100.0, 100.0, 100.0, 80.0, 60.0, 40.0, 20.0, 0.0, 100.0, 80.0, 100.0, 100.0, 100.0, 80.0, 60.0, 40.0, 100.0, 100.0, 100.0, 100.0,

### Parabolic Stop and Reverse
The parabolic SAR attempts to give the caller a direction of the trend

In [386]:
weekly_parabolic_sar = bulk_trend_indicators.parabolic_sar(data['High'], data['Low'], data['Close'], 5)
monthly_parabolic_sar = bulk_trend_indicators.parabolic_sar(data['High'], data['Low'], data['Close'], 20)
quarterly_parabolic_sar = bulk_trend_indicators.parabolic_sar(data['High'], data['Low'], data['Close'], 60)

print(weekly_parabolic_sar)

weekly_parabolic_sar.append(single_trend_indicators.parabolic_sar(
    data['High'][-4:] + latest_high, 
    data['Low'][-4:] + latest_low, 
    data['Close'][-4:] + latest_close,
    weekly_parabolic_sar[-1][0],
    weekly_parabolic_sar[-1][1],
    weekly_parabolic_sar[-1][2],
    weekly_parabolic_sar[-1][3],
))
monthly_parabolic_sar.append(single_trend_indicators.parabolic_sar(
    data['High'][-19:] + latest_high, 
    data['Low'][-19:] + latest_low, 
    data['Close'][-19:] + latest_close,
    monthly_parabolic_sar[-1][0],
    monthly_parabolic_sar[-1][1],
    monthly_parabolic_sar[-1][2],
    monthly_parabolic_sar[-1][3],
))
quarterly_parabolic_sar.append(single_trend_indicators.parabolic_sar(
    data['High'][-59:] + latest_high, 
    data['Low'][-59:] + latest_low, 
    data['Close'][-59:] + latest_close,
    quarterly_parabolic_sar[-1][0],
    quarterly_parabolic_sar[-1][1],
    quarterly_parabolic_sar[-1][2],
    quarterly_parabolic_sar[-1][3],
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if close[0] > close[-1]:
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif close[0] < close[-1]:
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/trend_indicators.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated.

[(3497.0042, 0.02, 3762.79, 'rising'), (3502.319916, 0.02, 3762.79, 'rising'), (3507.52931768, 0.02, 3762.79, 'rising'), (3519.6577449728, 0.04, 3810.74, 'rising'), (3540.2492802744323, 0.06, 3862.85, 'rising'), (3567.921337852478, 0.08, 3886.15, 'rising'), (3593.3796308242795, 0.08, 3886.15, 'rising'), (3624.5836677418515, 0.1, 3905.42, 'rising'), (3652.6673009676665, 0.1, 3905.42, 'rising'), (3683.7620248515464, 0.12000000000000001, 3911.79, 'rising'), (3711.125381869361, 0.12000000000000001, 3911.79, 'rising'), (3735.2051360450373, 0.12000000000000001, 3911.79, 'rising'), (3756.395319719633, 0.12000000000000001, 3911.79, 'rising'), (3775.0426813532767, 0.12000000000000001, 3911.79, 'rising'), (3791.4523595908836, 0.12000000000000001, 3911.79, 'rising'), (3780.2560764399777, 0.12000000000000001, 3698.15, 'falling'), (3801.6249472671802, 0.12000000000000001, 3958.33, 'rising'), (3829.604654649775, 0.14, 4001.48, 'rising'), (3858.303109905811, 0.16, 4008.97, 'rising'), (3888.9997501227

## Candle Indicators
Candle indicators are used on candle charts to attempt to give buy and sell signals.

In [387]:
from PyTechnicalIndicators.Bulk import candle_indicators as bulk_candle_indicators
from PyTechnicalIndicators.Single import candle_indicators as single_candle_indicators

### Bollinger Bands
Bollinger bands are very similar to moving average envelope bands, the only difference is that it adds or substracts the standard deviation to the moving average
A buy signal is when the price breaks the lower band, and sell signal is when the upper band is broken.
The default parameters are to use the moving average as the ma model and to use 2 standard deviations for the addition/substraction, and looks at 20 periods. But these can be changed by the caller.
The single function doesn't take a period but determines it from the length of the price list

In [388]:
bollinger_bands = bulk_candle_indicators.bollinger_bands(data['Typical Price'])

print(bollinger_bands)

bollinger_bands.append(single_candle_indicators.bollinger_bands(data['Typical Price'][-19:] + latest_typical_price))

[(3604.0574009609604, 3928.259932372372), (3620.987842310847, 3942.252491022486), (3633.314749245459, 3965.408250754539), (3643.065124411254, 3986.2008755887446), (3645.0604603481856, 4011.020206318481), (3659.633851371493, 4023.31248196184), (3683.084108701957, 4024.835891298043), (3701.2164972682153, 4030.8085027317843), (3710.387069793142, 4038.156263540191), (3710.9639169241914, 4052.410749742476), (3709.783308873971, 4070.887357792696), (3712.992853517625, 4088.00647981571), (3715.76691985579, 4095.8070801442086), (3719.012201114957, 4100.6671322183765), (3720.4670357742402, 4115.449297559094), (3729.6482926015774, 4133.439374065091), (3764.3855436397776, 4132.3747896935565), (3798.4308557836757, 4123.664477549658), (3826.159010273077, 4111.793323060256), (3847.8659429369545, 4101.43239039638), (3900.510593388476, 4067.5390732781916), (3904.5474430235945, 4065.830556976407), (3903.9918405656354, 4065.6308261010327), (3903.981073571591, 4072.0089264284097), (3904.331359075862, 4073

### Personalised Bollinger Bands

In [389]:
weekly_bband = bulk_candle_indicators.bollinger_bands(data['Typical Price'], 5, 'ema', 3)
monthly_bband = bulk_candle_indicators.bollinger_bands(data['Typical Price'], 20, 'ema', 3)
quarterly_bband = bulk_candle_indicators.bollinger_bands(data['Typical Price'], 60, 'ema', 3)

print(weekly_bband)

weekly_bband.append(single_candle_indicators.bollinger_bands(
    data['Typical Price'][-4:] + [latest_typical_price],
    'ema',
    3
))
monthly_bband.append(single_candle_indicators.bollinger_bands(
    data['Typical Price'][-19:] + [latest_typical_price],
    'ema',
    3
))
quarterly_bband.append(single_candle_indicators.bollinger_bands(
    data['Typical Price'][-59:] + [latest_typical_price],
    'ema',
    3
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/moving_averages.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_sum += prices[i] * alpha_power


[(3546.6098405300763, 3822.188674477824), (3578.9549067204034, 3798.0815545750174), (3635.151899766963, 3771.9893956516794), (3622.798417675389, 3849.031250571058), (3580.9245606266663, 3972.472785344899), (3592.7624121407835, 4022.64757206143), (3661.6040206255284, 3978.36880717858), (3746.507750387915, 3943.2518072740127), (3794.088019952503, 3926.024365513534), (3798.3843216682494, 3931.7757731184824), (3743.591245843021, 3941.7146309342315), (3598.3448398114165, 3996.9183829374), (3576.1498884012485, 3981.651407017395), (3614.4451403621506, 3945.984654266603), (3671.533085055976, 3913.837057124121), (3668.5759087870956, 3898.4593202808364), (3641.0867133898605, 4040.601027526413), (3629.829793671517, 4166.818831920902), (3652.664549013676, 4212.675988111128), (3685.3296876406503, 4235.911070653189), (3895.8836567569087, 4048.271351141987), (3897.7647702170657, 4023.3186737007877), (3898.6588984915866, 4020.3768992967257), (3892.2266919661433, 4016.8084423150603), (3906.255823888306

### Ichimoku Cloud
The Ichimoku cloud tries to show support/resistance levels as well as general trend direction.

The idea is that when span A is over span B there is potential for an uptrend, this is reinforced is the price is above the cloud.

The default periods for this are based on a 6 day work week, the conversion period defaults to 9, the base period defaults to 26, and the span b period defaults to 52. These can be changed by the caller.


In [390]:
ichimoku_cloud = bulk_candle_indicators.ichimoku_cloud(data['High'], data['Low'], data['Close'])

print(ichimoku_cloud)

ichimoku_cloud.append(single_candle_indicators.ichimoku_cloud(
    data['High'][-59:] + [latest_high],
    data['Low'][-59:] + [latest_low],
    data['Close'][-59:] + [latest_close]
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/candle_indicators.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return leading_span_a, leading_span_b, base_line, conversion_line, close[-base_period]


[(3920.925, 3796.27, 3932.725, 3909.125, 3965.34), (3897.0775, 3840.3199999999997, 3932.725, 3861.43, 3949.94), (3880.2124999999996, 3869.8050000000003, 3932.725, 3827.7, 4003.58), (3879.9399999999996, 3874.19, 3932.725, 3827.1549999999997, 4027.26), (3879.9399999999996, 3874.19, 3932.725, 3827.1549999999997, 4026.12), (3879.9399999999996, 3874.19, 3932.725, 3827.1549999999997, 3963.94), (3877.1, 3874.19, 3932.725, 3821.475, 3957.63), (3888.105, 3899.5550000000003, 3932.725, 3843.485, 4080.11), (3899.2, 3899.5550000000003, 3932.725, 3865.675, 4076.57), (3899.2, 3899.5550000000003, 3932.725, 3865.675, 4071.7), (3907.4624999999996, 3899.5550000000003, 3932.725, 3882.2, 3998.84), (3914.385, 3899.5550000000003, 3932.725, 3896.045, 3941.26), (3915.9325, 3899.5550000000003, 3932.725, 3899.14, 3933.92), (3920.8149999999996, 3899.5550000000003, 3932.725, 3908.9049999999997, 3963.51), (3920.8149999999996, 3899.5550000000003, 3932.725, 3908.9049999999997, 3934.38), (3922.6, 3899.5550000000003, 3

### Personalised Ichimoku Cloud
The Ichimoku cloud function can be personalised to adapt to the data the caller is using.

In [392]:
personalised_ichimoku_cloud = bulk_candle_indicators.ichimoku_cloud(data['High'], data['Low'], data['Close'], 5, 20, 60)

print(personalised_ichimoku_cloud)

personalised_ichimoku_cloud.append(single_candle_indicators.ichimoku_cloud(
    data['High'][-60:] + [latest_high],
    data['Low'][-60:] + [latest_low],
    data['Close'][-60:] + [latest_close],
    5, 20, 60
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/candle_indicators.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return leading_span_a, leading_span_b, base_line, conversion_line, close[-base_period]


[(3902.5874999999996, 3796.27, 3932.725, 3872.45, 3934.38), (3904.6099999999997, 3840.3199999999997, 3932.725, 3876.495, 3990.56), (3909.4849999999997, 3869.8050000000003, 3932.725, 3886.245, 4019.65), (3906.3925, 3874.19, 3909.125, 3903.66, 3995.32), (3912.42, 3874.19, 3884.22, 3940.62, 3895.75), (3918.14, 3874.19, 3889.9399999999996, 3946.34, 3852.36), (3930.4649999999997, 3874.19, 3889.9399999999996, 3970.99, 3817.66), (3920.2025, 3899.5550000000003, 3889.9399999999996, 3950.465, 3821.62), (3920.2025, 3899.5550000000003, 3889.9399999999996, 3950.465, 3878.44), (3932.1625, 3899.5550000000003, 3901.8999999999996, 3962.425, 3822.39), (3936.235, 3899.5550000000003, 3910.045, 3962.425, 3844.82), (3936.235, 3899.5550000000003, 3910.045, 3962.425, 3829.25), (3950.445, 3899.5550000000003, 3921.175, 3979.715, 3783.22), (3982.9525000000003, 3899.5550000000003, 3944.27, 4021.635, 3849.28), (3982.9525000000003, 3899.5550000000003, 3944.27, 4021.635, 3839.5), (3982.9525000000003, 3899.5550000000

## Volatility

In [393]:
from PyTechnicalIndicators.Bulk import volatility_indicators as bulk_volatility_indicators
from PyTechnicalIndicators.Single import volatility_indicators as single_volatility_indicators

### Average True Range
The average true range (ATR) gives the caller an idea of the range or spread of the prices over a given period. The higher the range, the more volatile the price.

In the single function the length of the passed prices is used to determine the period. There are also two functions for the average true range, one where the previous true range is known `average_true_range` and one where the initial is not known `average_true_range_initial` which should only be used to initialize the ATR list.

In [394]:
# TODO: the close price needs to be shifted one as it should be t-1 close
weekly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
monthly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
quarterly_atr = bulk_volatility_indicators.average_true_range(
    data['High'],
    data['Low'],
    data['Close'],
    5
)

print(weekly_atr)

weekly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 5))
monthly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 20))
quarterly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 60))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/volatility_indicators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sum_true_range += other_indicators.true_range(high[i], low[i], previous_close[i])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/volatility_indicators.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  atr_list.append(volatility_indicators.average_true_range(high[i + period - 1], low[i + period - 1], close[i + period - 1], atr_list[-1], period))


[103.1119999999999, 98.4015999999999, 100.81527999999989, 94.47022399999985, 88.25817919999984, 83.02254335999986, 77.65003468799986, 81.55202775039986, 71.35162220031992, 70.67929776025589, 83.69543820820476, 77.44435056656383, 79.45548045325106, 73.41438436260084, 73.35550749008065, 73.48040599206453, 78.47232479365164, 74.10985983492128, 69.80188786793697, 70.97551029434959, 62.53040823547967, 59.58232658838373, 56.44786127070695, 50.89028901656553, 50.51223121325243, 47.43378497060201, 40.599027976481565, 43.77922238118525, 42.847377904948175, 62.58390232395858, 59.99512185916693, 58.766097487333525, 60.60487798986683, 65.10790239189353, 59.06432191351489, 54.92345753081194, 52.734766024649545, 53.269812819719604, 64.20185025577565, 68.98348020462055, 70.97078416369645, 69.37662733095716, 66.46530186476576, 61.696241491812586, 59.42299319345015, 65.2923945547602, 61.99191564380815, 56.27953251504657, 58.53162601203725, 57.373300809629846, 53.80064064770384, 59.866512518163084, 59.3

/tmp/ipykernel_1851/2804827742.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  weekly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 5))
/tmp/ipykernel_1851/2804827742.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  monthly_atr.append(single_volatility_indicators.average_true_range(latest_high, latest_low, data['Close'][-1], weekly_atr[-1], 20))
/tmp/ipykernel_1851/2804827742.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent w

### Ulcer Index
The Ulcer index (UI) calculates volatility by looking at the difference between the period high and latest price. Peaks that are higher than the average denote high volatility.

The default period is 14 days but can be changed by the caller.

The single function determines the period from the list of prices.

In [395]:
weekly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 5)
monthly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 20)
quarterly_ui = bulk_volatility_indicators.ulcer_index(data['Close'], 60)

print(weekly_ui)

weekly_ui.append(data['Close'][-4:] + [latest_close])
monthly_ui.append(data['Close'][-19:] + [latest_close])
quarterly_ui.append(data['Close'][-59:] + [latest_close])

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/volatility_indicators.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  percentage_drawdown = ((close_prices[i] - period_high) / period_high) * 100


[1.0994910405290432, 0.716659871774382, 0.716659871774382, 0.716659871774382, 0.35557690156281757, 0.33038860275306736, 0.6853014166710237, 0.6853014166710237, 0.7620851127567259, 0.6714061997863036, 1.7330035934475505, 2.704972452051144, 2.4847433135964585, 2.2487713364346886, 0.4734193803451457, 0.929214922927153, 0.929214922927153, 0.929214922927153, 1.011502448034867, 0.39984680489030927, 0.5532506745328108, 0.7588221502538156, 0.6926089180618324, 0.8360102305884016, 0.22193106088641662, 0.17368219044771427, 0.174142934154315, 0.7032611222741048, 1.0451993327922717, 1.0451993327922717, 1.0282633774592043, 0.12276304374503025, 0.8963830079952192, 1.7662709207365652, 2.3199997021601173, 2.52885885630429, 1.271852385060062, 0.3390698221172422, 0.3286816997286199, 0.4257976128477727, 1.4047982106840031, 2.3318624325993413, 3.2384929815674814, 3.3967332339470913, 1.3473477569916437, 0.8410213945184307, 0.7536484499313767, 0.9432389065551829, 1.4474852193813514, 0.7390393241434239, 0.747

### Welles Volatility Index
Welles developed his volatility index (VI) to observe volatility of assets.

The single function takes the latest value of the volatility index as a parameter if it is available.


In [396]:
# TODO: Go to book to get better description, the internet has very little
weekly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    5
)
monthly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    20
)
quarterly_vi = bulk_volatility_indicators.volatility_index(
    data['High'],
    data['Low'],
    data['Close'],
    60
)

print(weekly_vi)

weekly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    weekly_vi[-1]
))
monthly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    monthly_vi[-1]
))
quarterly_vi.append(single_volatility_indicators.volatility_index(
    latest_high,
    latest_low,
    latest_close,
    5,
    quarterly_vi[-1]
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/volatility_indicators.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vi.append(volatility_indicators.volatility_index(high[i], low[i], close[i], period, 0))
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/volatility_indicators.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vi.append(volatility_indicators.volatility_index(high[i], low[i], close[i], period, vi[-1]))


[38.765999999999984, 57.476800000000026, 56.19744000000001, 60.20995199999995, 60.58196159999991, 64.37756927999992, 73.5960554239999, 72.69484433919986, 70.83787547135987, 69.08630037708788, 66.50104030167027, 72.63283224133619, 64.21626579306898, 64.97101263445515, 79.12881010756416, 73.79104808605135, 76.53283846884108, 71.07627077507286, 71.48501662005826, 71.98401329604661, 77.2752106368373, 73.15216850946982, 69.03573480757579, 70.36258784606065, 62.04007027684852, 59.19005622147881, 56.134044977183024, 50.63923598174639, 50.311388785397114, 47.27311102831776, 40.47048882265416, 43.67639105812333, 42.76511284649864, 62.51809027719895, 59.942472221759225, 58.72397777740737, 60.5711822219259, 65.08094577754079, 59.0427566220327, 54.90620529762619, 52.720964238100954, 53.25877139048073, 64.19301711238455, 68.97641368990767, 70.96513095192616, 69.37210476154092, 66.46168380923277, 61.69334704738619, 59.42067763790904, 65.29054211032732, 61.99043368826185, 56.27834695060953, 58.530677

### Welles Volatility System
Same go to book for more info

In [397]:
weekly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    5,
    2
)
monthly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    20,
    2
)
quarterly_vs = bulk_volatility_indicators.volatility_system(
    data['High'],
    data['Low'],
    data['Close'],
    60,
    2
)

print(weekly_vs)

weekly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-4:] + [latest_high],
    data['Low'][-4:] + [latest_low],
    data['Close'][-4:] + [latest_close],
    5,
    2,
    weekly_vs[-1]
))
monthly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-19:] + [latest_high],
    data['Low'][-19:] + [latest_low],
    data['Close'][-19:] + [latest_close],
    20,
    2,
    monthly_vs[-1]
))
quarterly_vs.append(single_volatility_indicators.volatility_system(
    data['High'][-59:] + [latest_high],
    data['Low'][-59:] + [latest_low],
    data['Close'][-59:] + [latest_close],
    59,
    2,
    quarterly_vs[-1]
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/volatility_indicators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sum_true_range += other_indicators.true_range(high[i], low[i], previous_close[i])
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/volatility_indicators.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  atr = average_true_range(high[-1], low[-1], close[-1], atr_initial, period)
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalI

[(3719.98, 196.8031999999998, 3523.1768, 98.4015999999999), (3752.75, 201.63055999999978, 3551.1194400000004, 100.81527999999989), (3797.34, 188.9404479999997, 3608.3995520000003, 94.47022399999985), (3859.11, 176.5163583999997, 3682.5936416000004, 88.25817919999984), (3859.11, 166.04508671999972, 3693.0649132800004, 83.02254335999986), (3859.11, 155.30006937599973, 3703.8099306240006, 77.65003468799986), (3901.06, 163.10405550079972, 3737.9559444992, 81.55202775039986), (3901.06, 142.70324440063985, 3758.35675559936, 71.35162220031992), (3901.06, 141.35859552051178, 3759.7014044794882, 70.67929776025589), (3759.69, 167.39087641640953, 3927.0808764164094, 83.69543820820476), (3719.89, 154.88870113312765, 3874.7787011331275, 77.44435056656383), (3770.55, 158.9109609065021, 3929.4609609065024, 79.45548045325106), (3806.8, 146.82876872520168, 3953.628768725202, 73.41438436260084), (3828.11, 146.7110149801613, 3974.8210149801616, 73.35550749008065), (3748.57, 146.96081198412907, 3895.53081

## Correlation

In [398]:
from PyTechnicalIndicators.Bulk import correlation_indicators as bulk_correlation_indicators
from PyTechnicalIndicators.Single import correlation_indicators as single_correlation_indicators

### Correlate asset prices
A simple function that allows the caller to correlate two asset prices for a given period.

A value of 1 means that assets are perfectly correlated, the prices move in the exact same way. 
A value of -1 means that they are perfectly uncorrelated, the prices move in opposite directions.

For the single function the period is determined from the length of prices

In [399]:
# TODO: get another asset to do the correlation here
correlation = bulk_correlation_indicators.correlate_asset_prices(data['Typical Price'], data['Typical Price'], 5)

print(correlation)

correlation.append(single_correlation_indicators.correlate_asset_prices(
    data['Typical Price'][-4:] + [latest_typical_price], 
    data['Typical Price'][-4:] + [latest_typical_price]
))

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/correlation_indicators.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  asset_a_avg_return = price_asset_a[i] - asset_a_avg_price
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Single/correlation_indicators.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  asset_b_avg_return = price_asset_b[i] - asset_b_avg_price


[0.9999999999999998, 0.9999999999999998, 1.0, 1.0, 1.0, 0.9999999999999997, 1.0, 0.9999999999999998, 0.9999999999999998, 1.0000000000000002, 1.0000000000000002, 0.9999999999999998, 1.0000000000000002, 1.0000000000000002, 1.0, 0.9999999999999999, 1.0, 1.0000000000000002, 1.0, 1.0, 1.0, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0, 1.0, 1.0000000000000002, 1.0, 0.9999999999999999, 1.0000000000000002, 0.9999999999999996, 1.0, 1.0, 0.9999999999999998, 0.9999999999999998, 1.0, 1.0000000000000002, 1.0, 1.0000000000000002, 0.9999999999999998, 1.0, 0.9999999999999998, 1.0, 0.9999999999999997, 1.0, 1.0, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0, 1.0, 0.9999999999999997, 1.0, 1.0, 1.0, 1.0000000000000002, 0.9999999999999999, 0.9999999999999998, 1.0000000000000002, 1.0, 1.0000000000000002, 0.9999999999999998, 1.0, 1.0, 1.0000000000000002, 1.0, 0.9999999999999996, 1.0000000000000002, 1.0000000000000002, 1.0, 1.0000000000000002, 1.0, 0.9999999999999999, 1.0, 1.0, 0.9999999

## Support and resistance indicators 

In [400]:
from PyTechnicalIndicators.Bulk import support_resistance_indicators as bulk_support_resistance_indicators
from PyTechnicalIndicators.Single import support_resistance_indicators as single_support_resistance_indicators

### Fibonacci retracement
Calculates positions based on the Fibonacci sequence.

In [401]:
fibonacci_retracement = bulk_support_resistance_indicators.fibonacci_retracement(data['Typical Price'])

print(fibonacci_retracement)

fibonacci_retracement.append(single_support_resistance_indicators.fibonacci_retracement(latest_typical_price))

[(3615.633333333333, 4468.922799999999, 4996.805266666666, 5423.45, 5850.094733333333, 6377.9772, 7231.266666666666), (3624.9166666666665, 4480.397, 5009.634833333333, 5437.375, 5865.1151666666665, 6394.353, 7249.833333333333), (3668.7766666666666, 4534.60796, 5070.249353333333, 5503.165, 5936.080646666667, 6471.72204, 7337.553333333333), (3723.1, 4601.7516, 5145.324199999999, 5584.65, 6023.9758, 6567.5484, 7446.2), (3696.75, 4569.183, 5108.9085, 5545.125, 5981.3415, 6521.067, 7393.5), (3686.0733333333337, 4555.98664, 5094.153346666667, 5529.110000000001, 5964.066653333334, 6502.233360000001, 7372.146666666667), (3719.353333333333, 4597.12072, 5140.146306666666, 5579.03, 6017.913693333333, 6560.93928, 7438.706666666666), (3783.2433333333333, 4676.08876, 5228.442286666666, 5674.865, 6121.287713333334, 6673.64124, 7566.486666666667), (3840.4666666666667, 4746.8168, 5307.524933333333, 5760.7, 6213.875066666667, 6774.5832, 7680.933333333333), (3846.94, 4754.81784, 5316.471079999999, 5770.4

### Pivot points
Calculates the pivot as well as the primary and secondary support ad resistance levels.

In [402]:
pivot_points = bulk_support_resistance_indicators.pivot_points(data['High'], data['Low'], data['Close'])

print(pivot_points)

pivot_points.append(single_support_resistance_indicators.pivot_points(latest_high, latest_low, latest_close))

[(3615.633333333333, 3545.8566666666666, 3739.6866666666665, 3421.8033333333333, 3809.463333333333), (3624.9166666666665, 3537.833333333333, 3670.153333333333, 3492.5966666666664, 3757.2366666666667), (3668.7766666666666, 3647.8233333333333, 3698.903333333333, 3617.6966666666667, 3719.8566666666666), (3723.1, 3683.41, 3759.6699999999996, 3646.84, 3799.3599999999997), (3696.75, 3664.92, 3726.99, 3634.6800000000003, 3758.8199999999997), (3686.0733333333337, 3636.1466666666674, 3715.7066666666674, 3606.513333333334, 3765.6333333333337), (3719.353333333333, 3680.816666666666, 3791.286666666666, 3608.883333333333, 3829.823333333333), (3783.2433333333333, 3755.746666666667, 3824.8366666666666, 3714.1533333333336, 3852.333333333333), (3840.4666666666667, 3818.0833333333335, 3881.4933333333333, 3777.056666666667, 3903.8766666666666), (3846.94, 3807.73, 3869.81, 3784.86, 3909.02), (3823.6800000000003, 3787.4100000000008, 3843.5700000000006, 3767.5200000000004, 3879.84), (3871.58, 3837.74, 3934.

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/support_resistance_indicators.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pp.append(support_resistance_indicators.pivot_points(highs[i], lows[i], close[i]))


## Other indicators


In [403]:
from PyTechnicalIndicators.Bulk import other_indicators as bulk_other_indicators
from PyTechnicalIndicators.Single import other_indicators as single_other_indicators

### Value added index
Calculates the value added for a given investment if it had been bought at the beginning of the passed in list of prices.

The starting investment defaults to 1000 but can be changed by the caller. 
The passed in price doesn't have to (and probably shouldn't) be the entire price history, just when the caller invested in or was planning on in the stock studied.

The single function can be passed in the previous value added index to continue the calculations.

In [404]:
value_added_index = bulk_other_indicators.value_added_index(data['Typical Price'])

print(value_added_index)

value_added_index.append(single_other_indicators.value_added_index(data['Typical Price'][-1], latest_typical_price, value_added_index[-1]))

[10283.333333333303, 461310.33333333326, 25521225.341111075, -646963062.3971634, 6260445900.462967, 214608085467.86606, 13925918666009.898, 810813404463983.4, 6059478842694189.0, -1.3488399903837122e+17, -6.595827552976303e+18, -3.77061475111814e+19, 1.7847576488625062e+20, -1.1653277608639625e+22, 9.36845831217236e+23, 3.438224200567239e+25, 1.2918554396264772e+27, 3.933269195182754e+28, -2.0951213913006888e+30, -3.258612137269671e+32, -1.8209124623062957e+34, 8.260872870663248e+34, 1.490261465867647e+36, -3.7018094812151214e+37, 1.0585941179781729e+39, 2.7092952126121283e+40, -2.973903045043857e+41, -1.2298080392271402e+43, -3.950143421997561e+44, -3.1864490270781883e+45, 1.5498888067708557e+47, -2.941172325648795e+48, -2.252447806059387e+50, -9.922032585691537e+51, 1.526008611679324e+53, -7.122899529782007e+54, 4.074773391003947e+56, -5.974976049008634e+57, -1.2419983547206063e+59, 1.0801245691554728e+60, 2.704991962688425e+61, 1.8039591399169e+63, -5.767257370314294e+64, 5.34740103

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/other_indicators.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vai_list = [other_indicators.value_added_index(prices[0], prices[1], starting_investment)]
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Bulk/other_indicators.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vai_list.append(other_indicators.value_added_index(prices[i], prices[i+1], vai_list[-1]))
/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyT

### Welles Other indicators
A series of other indicators from Welles that are mostly used by other functions.

These will probably be moved later on, so this should probably be ignored.

# Chart Patterns
The chart patterns are a collection of indicators made to be displayed on the OHLC charts to shows trends.

## Peaks

In [405]:
from PyTechnicalIndicators.Chart_Patterns import peaks

### Get peaks
Simple function that gets all the highest point for a given period.

If you choose a period of 20 it will get the peak for the first 20 periods, then move on to the next 20 periods (which could have the same peak) and returns the peak value and its index in the list.

Period defaults to 5. It is recommended to use the highs if planning on plotting against an ohlc chart.

Purpose is to help identify where the peaks are for a period so that they can be highlighted on a chart.

In [406]:
weekly_peaks = peaks.get_peaks(data['High'] + [latest_high], 5)
monthly_peaks = peaks.get_peaks(data['High'] + [latest_high], 20)
quarterly_peaks = peaks.get_peaks(data['High'] + [latest_high], 60)

print(weekly_peaks)

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Chart_Patterns/peaks.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if sub_prices[i] == peak:


[(8145.46, 3), (8193.41, 7), (8245.52, 8), (8268.82, 9), (8288.09, 11), (8294.46, 13), (8277.11, 14), (8242.07, 18), (8341.0, 20), (8384.15, 21), (8391.64, 22), (8411.51, 23), (8388.55, 28), (8416.45, 29), (8416.69, 30), (8462.78, 33), (8483.18, 34), (8463.15, 35), (8435.119999999999, 36), (8384.18, 37), (8483.630000000001, 42), (8436.43, 43), (8341.04, 44), (8273.58, 45), (8272.49, 48), (8240.86, 53), (8261.130000000001, 55), (8288.86, 58), (8333.24, 59), (8352.74, 61), (8380.43, 62), (8386.619999999999, 63), (8398.06, 64), (8421.98, 68), (8444.24, 71), (8476.880000000001, 72), (8531.619999999999, 75), (8578.11, 76), (8565.029999999999, 77), (8559.21, 79), (8542.44, 84), (8530.779999999999, 85), (8519.21, 86), (8464.18, 87), (8435.02, 88), (8410.970000000001, 90), (8400.720000000001, 92), (8430.96, 96), (8461.16, 97), (8432.67, 98), (8400.48, 100), (8347.130000000001, 105), (8391.75, 108), (8422.16, 109), (8413.26, 114), (8440.52, 115), (8493.42, 116), (8510.33, 117), (8515.8, 118), (

## Valleys

In [407]:
from PyTechnicalIndicators.Chart_Patterns import valleys

### Get Valleys
Get valleys gets the lowest point for a given period.

Period defaults to 5 but should be determined by the caller. The function gets the lowest point for the period over the length of passed prices. Which it then returns with its index.

Purpose is to help identify where the valleys are for a period so that they can be highlighted on a chart.

In [408]:
weekly_valleys = valleys.get_valleys(data['Low'] + [latest_low], 5)
monthly_valleys = valleys.get_valleys(data['Low'] + [latest_low], 20)
quarterly_valleys = valleys.get_valleys(data['Low'] + [latest_low], 60)

print(weekly_valleys)

/home/j/Projects/PyTechnicalIndicators_Examples/venv/lib/python3.10/site-packages/PyTechnicalIndicators/Chart_Patterns/valleys.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if sub_prices[i] == valley:


[(7861.92, 0), (7950.02, 1), (8008.99, 2), (8017.76, 6), (8111.99, 7), (8169.780000000001, 8), (8174.13, 10), (8129.02, 14), (8068.49, 15), (8079.18, 16), (8114.5599999999995, 19), (8230.23, 20), (8276.880000000001, 25), (8303.68, 27), (8307.99, 32), (8288.73, 37), (8293.02, 38), (8303.380000000001, 40), (8249.79, 44), (8198.25, 45), (8170.38, 46), (8165.96, 47), (8134.83, 49), (8151.120000000001, 52), (8164.67, 55), (8172.76, 57), (8179.9, 58), (8247.630000000001, 60), (8296.93, 65), (8255.880000000001, 66), (8268.2, 67), (8319.4, 70), (8383.630000000001, 71), (8385.89, 73), (8390.78, 74), (8407.54, 75), (8458.73, 79), (8440.01, 81), (8431.130000000001, 82), (8418.29, 87), (8365.53, 88), (8347.24, 89), (8339.53, 90), (8313.42, 91), (8309.39, 94), (8298.5, 95), (8279.04, 100), (8216.66, 101), (8179.200000000001, 102), (8208.58, 104), (8234.45, 105), (8271.61, 106), (8279.5, 111), (8321.87, 113), (8369.87, 114), (8402.44, 115), (8426.52, 116), (8440.18, 120), (8442.89, 121), (8457.28, 1

## Chart Trends
Chart trends brings together the peak and valley functions to try and break down the trends on a chart

In [409]:
from PyTechnicalIndicators.Chart_Patterns import chart_trends

ModuleNotFoundError: No module named 'src'

### Get Trend Line
The get trend line is a function that calculates the best line that passes through all provided points.
It is mainly intended for internal use by the functions that will follow but can be used if given the correct parameters. 

In [ ]:
weekly_high_trendline = chart_trends.get_trend_line(weekly_valleys)
print(weekly_high_trendline)

### Get Peak Trend
The get peak trend function merges the get peaks and get trend function.

It gets the peaks then finds the best fitting line for all the points over a given period

It returns the slope and the intercept as a tuple so that they can be plotted on a chart.

In [ ]:
weekly_peak_trend = chart_trends.get_peak_trend(data['High'] + [latest_high], 5)
monthly_peak_trend = chart_trends.get_peak_trend(data['High'] + [latest_high], 20)
quarterly_peak_trend = chart_trends.get_peak_trend(data['High'] + [latest_high], 60)

print(weekly_peak_trend)

weekly_peak_points = [(i*weekly_peak_trend[i][0]) + weekly_peak_trend[i][1] for i in range(0, len(weekly_peak_trend))]
monthly_peak_points = [(i*monthly_peak_trend[i][0]) + monthly_peak_trend[i][1] for i in range(0, len(monthly_peak_trend))]
quarterly_peak_points = [(i*quarterly_peak_trend[i][0]) + quarterly_peak_trend[i][1] for i in range(0, len(quarterly_peak_trend))]















### Get Valley Trend
The get valley trend function merges the get valleys and get trend function.

It gets the valleys then finds the best fitting line for all the points over a given period

It returns the slope and the intercept as a tuple so that they can be plotted on a chart.

In [ ]:
weekly_valley_trend = chart_trends.get_valley_trend(data['Low'] + [latest_low], 5)
monthly_valley_trend = chart_trends.get_valley_trend(data['Low'] + [latest_low], 20)
quarterly_valley_trend = chart_trends.get_valley_trend(data['Low'] + [latest_low], 60)

print(weekly_valley_trend)

weekly_valley_points = [(i*weekly_valley_trend[i][0]) + weekly_valley_trend[i][1] for i in range(0, len(weekly_valley_trend))]
monthly_valley_points = [(i*monthly_valley_trend[i][0]) + monthly_valley_trend[i][1] for i in range(0, len(monthly_valley_trend))]
quarterly_valley_points = [(i*quarterly_valley_trend[i][0]) + quarterly_valley_trend[i][1] for i in range(0, len(quarterly_valley_trend))]

### Get overall trend
The get overall trend function returns the trend for passed in list of prices.

It is recommended to use the typical price as it takes into account the high, low, and close of the day, but any price can be used.

It is worth keeping in mind that the overall trend function will try to find the best fitting line for everything single point so very volatile prices could return some unexpected results.

In [ ]:
overall_trend = chart_trends.get_overall_trend(data['Typical Price'])

print(overall_trend)

overall_trend_points = [(i*overall_trend[i][0]) + overall_trend[i][1] for i in range(0, len(overall_trend))]

### Break Down Trends
The break down trends function gives the caller information on where a trend starts and ends, and what the slope and intercept are for that trend.

This allows the caller to use those points and chart them, so that the different starts and ends of trends are visible.



In [ ]:
trends_low_sensitivity = chart_trends.break_down_trends(data['Typical Price'], 1)
trends_default_sensitivity = chart_trends.break_down_trends(data['Typical Price'])
trends_high_sensitivty = chart_trends.break_down_trends(data['Typical Price'], 5)


# THINGS TO FIX
* In here the negative positions need to be debugged to confirm that the correct amount gets sent in